IMPORTING NECESSARY LIBRARIES

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
#We will use arxiv to extract abstracts from pdfs


!pip install arxiv


     |████████████████████████████████| 81kB 3.7MB/s 
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-cp37-none-any.whl size=6067 sha256=3fb520a665138fa3936c5ab8c014668486eb4039073f3df4867f7be6cd6e1c76
  Stored in directory: /root/.cache/pip/wheels/f1/80/5a/444ba08a550cdd241bd9baf8bae44be750efe370adb944506a
Successfully built sgmllib3k


In [ ]:
import arxiv
import os
import json

In [ ]:
!pip install tika

  Created wheel for tika: filename=tika-1.24-cp37-none-any.whl size=32885 sha256=62fb3c88880cb6ecf055ae18cf25a98b6383faa8e95ed30509b8fba063e0f536
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika


In [ ]:
from tika import parser
import os

In [ ]:
from os import listdir
from os.path import isfile, join


In [ ]:
def Reverse(lst): 
    return [ele for ele in reversed(lst)] 
      
 


CLEANING AND CALCULATING THE EMBEDDINGS

In [ ]:
#we will give path of the drive where there are pdfs

path = '/content/drive/MyDrive/100_2/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from os import listdir
from os.path import isfile, join
onlyfiles2 = [f for f in listdir(path) if isfile(join(path, f))]

In [ ]:
onlyfiles2

['summarization (42).pdf',
 'summarization (43).pdf',
 'summarization (44).pdf',
 'summarization (45).pdf',
 'summarization (46).pdf',
 'summarization (47).pdf',
 'summarization (48).pdf',
 'summarization (49).pdf',
 'summarization (50).pdf',
 'sentiment_analysis (35).pdf',
 'sentiment_analysis (36).pdf',
 'sentiment_analysis (37).pdf',
 'sentiment_analysis (38).pdf',
 'sentiment_analysis (39).pdf',
 'sentiment_analysis (40).pdf',
 'sentiment_analysis (41).pdf',
 'sentiment_analysis (42).pdf',
 'sentiment_analysis (43).pdf',
 'sentiment_analysis (44).pdf',
 'document_classification (6).pdf',
 'document_classification (7).pdf',
 'document_classification (8).pdf',
 'document_classification (9).pdf',
 'document_classification (10).pdf',
 'document_classification (30).pdf',
 'document_classification (31).pdf',
 'document_classification (32).pdf',
 'document_classification (33).pdf',
 'document_classification (34).pdf',
 'document_classification (35).pdf',
 'document_classification (36).pdf

In [ ]:
#this is the code to extract abstracts/ titles from a pdf, We will simply first separate every line of the pdf and then find the Abstract and the paper name (title). Also we can find further stuff

myabstract=[]
topic_names=[]
file_names=[]
for j in onlyfiles2:
  file_names.append(j)
  parsed_pdf = parser.from_file(os.path.join(path, j))
  data = parsed_pdf['content']
  
  # Printing of content  
  with open('text.txt','w+') as pr:
    for key in parsed_pdf['content']:
      
      pr.write(key)
  strt=""
  with open('text.txt','r+') as f:
    strt+=f.read()
  contents = strt.split('\n')
  indexes=[]
  try:
    for i in range(len(contents)):
      if contents[i].find('Abstract') >= 0 or contents[i].find('ABSTRACT') >= 0:
        indexes.append(i)
    #indexes.append(contents.index('Abstract'))
    # for i in range(len(contents)):
    #   if contents[i].find('INTRODUCTION') >= 0 or contents[i].find('Introduction') >= 0:
    #     indexes.append(i)
    
    mydata = contents[indexes[0]:indexes[0]+300]
    mydata = '\n'.join(mydata)
    myabstract.append(mydata)
    

    for i in contents:
      if len(i)>1:
        topic_names.append(i)
        file_names.append(j)
        break
  except:
    pass

  
dict={}
#dict['file'] = file_list
dict['topic'] = topic_names
dict['abstract'] = myabstract

2021-04-08 14:27:29,379 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2021-04-08 14:27:30,379 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2021-04-08 14:27:30,796 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [ ]:
#from the above code we get a dictionary and then convert it into a dataframe

data2 = pd.DataFrame(dict)

In [ ]:
#Use this line of code separately. (If we are using any third party data) 

data2 = pd.read_csv('test_set_3.csv')

In [ ]:
data2.head()

,Unnamed: 0,title,abstract
0,0,(Probably) Concave Graph Matching,In this paper we address the graph matching pr...
1,1,3DAware Scene Manipulation via Inverse Graphics,"We aim to obtain an interpretable, expressive,..."
2,2,A Block Coordinate Ascent Algorithm for MeanVa...,Risk management in dynamic decision problems i...
3,3,A Deep Bayesian Policy Reuse Approach Against ...,"In multiagent domains, coping with non-station..."
4,4,A flexible model for training action localizat...,Spatio-temporal action detection in videos is ...


In [ ]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemma = WordNetLemmatizer()

In [ ]:
# This code cleans the abstracts with necessary libraries

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text
    #text = re.sub(r'http\S+','',text)
    text = re.sub('[^a-zA-Z]',' ',text)
    #text = re.sub(r'\s{2}','',text)
    text = str(text).lower()
    if '.' in str(text):
      text = str(text).replace('.',' ')
    if ',' in str(text):
      text = str(text).replace(',',' ')
    text = re.sub(r'abst\S+','',text)
    text = word_tokenize(text)
    text = [item for item in text if item not in stop_words]
    text = [i for i in text if len(i) > 2]
    # text = [stemmer.stem(i) for i in text]
    text = [lemma.lemmatize(word=w,pos='v') for w in text]
    text = ' '.join(text)

    
    return text

In [ ]:
data2['cleaned_text'] = data2.iloc[:,2].apply(clean_text)

In [ ]:
data2['cleaned_text']

0      paper address graph match problem follow recen...
1      aim obtain interpretable expressive disentangl...
2      risk management dynamic decision problems prim...
3      multiagent domains cop non stationary agents c...
4      spatio temporal action detection videos typica...
                             ...                        
287    introduce new efficient principled backpropaga...
288    infer intent observe behavior study extensivel...
289    give rigorous analysis statistical behavior gr...
290    humans make repeat choices among options imper...
291    object orient representations reinforcement le...
Name: cleaned_text, Length: 292, dtype: object

In [ ]:
len(data2)

292

In [ ]:
data_list2 = []
for i in data2['cleaned_text']:
  data_list2.append(i)

In [ ]:
topic_list2 = []
for i in data2['title']:
  topic_list2.append(i)

In [ ]:
#finding abstracts which are of very small length and then removing them from the data

count=0
my_list=[]
for i in data_list2:
  if len(i) < 40:
    my_list.append(count)
    count+=1
  else:
    count+=1

In [ ]:
my_list

[]

In [ ]:
data_list2 = [i for j, i in enumerate(data_list2) if j not in my_list]

In [ ]:
topic_list2 = [i for j, i in enumerate(topic_list2) if j not in my_list]

In [ ]:
data_list2

['paper address graph match problem follow recent work cite zaslavskiy path vestner analyze generalize idea concave relaxations introduce concepts emph conditionally concave emph probably conditionally concave energies polytopes show encapsulate many instance graph match problem include match euclidean graph graph surface prove local minima probably conditionally concave energies general match polytopes doubly stochastic high probability extreme point match polytope permutations',
 'aim obtain interpretable expressive disentangle scene representation contain comprehensive structural textural information object previous scene representations learn neural network often uninterpretable limit single object lack knowledge work propose scene render network sdn address issue integrate disentangle representations semantics geometry appearance deep generative model scene encoder perform inverse graphics translate scene structure object wise representation decoder two components differentiable s

In [ ]:
len(data_list2)

292

In [ ]:
#Converting every data point into a fixed length so at the time of calculating the embeddings we won't have any issue

data_list_fin2=[]
for i in data_list2:
  data_list_fin2.append(i[:2000])

In [ ]:
!pip install sentence_transformers

     |████████████████████████████████| 81kB 3.6MB/s 
     |████████████████████████████████| 2.3MB 7.3MB/s 
     |████████████████████████████████| 1.2MB 25.1MB/s 
     |████████████████████████████████| 901kB 38.6MB/s 
     |████████████████████████████████| 3.3MB 32.6MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.1-cp37-none-any.whl size=123338 sha256=386b40a9d116077cec706dbdbb66cf50e3fa5dfc87d0b821c69b446ea3f81990
  Stored in directory: /root/.cache/pip/wheels/5e/89/29/45e45adc162b50f97f71801e8b07947c9cfe2b3ae7dbf37896
Successfully built sentence-transformers


In [ ]:
# from sentence_transformers import SentenceTransformer, util

# model = SentenceTransformer('paraphrase-distilroberta-base-v1')



# #Compute embeddings
# embeddings = model.encode(data_list_fin, convert_to_tensor=False)


# from sklearn.cluster import KMeans
# num_clusters = 7
# # Define kmeans model
# clustering_model = KMeans(n_clusters=num_clusters)
# # Fit the embedding with kmeans clustering.
# clustering_model.fit(embeddings)
# # Get the cluster id assigned to each news headline.
# clusterr = clustering_model.get_params
# cluster_assignment = clustering_model.labels_

# for i in range(len(data_list_fin)):
#   print(data_list_fin[i])
#   print('\t')
#   print(cluster_assignment[i])

In [ ]:
# I have trained a model (SBERT) on the same kind of dataset. We can import it and name it as "model" and comment the next two lines but not the last one. 

from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-distilroberta-base-v1')

#Compute embeddings
embeddings_test = model.encode(data_list_fin2, convert_to_tensor=False)

In [ ]:
len(embeddings_test)

292

In [ ]:
# from sentence_transformers import SentenceTransformer, util

# model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [ ]:
# First save the embeddings_test and then import it again, so the code will not break afterwards

np.savetxt("embeddings_test.csv", embeddings_test, delimiter=",")

In [ ]:
# now importing embeddings (created before hand 3270) and embeddings_test 

embeddings = pd.read_csv("embeddings_final.csv",header=None)
#embeddings_test = pd.read_csv("embeddings_test.csv",header=None)

CALCULATIONS FOR NEW CLUSTERS

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
#At first I just hard coded the sublist length, afterwards I will make it dynamic

length = 7
length2 = len(embeddings_test)
cosine_list = []
for i in range(length2):
  cosine_list.append([])
  for j in range(length):
    cosine_list[i].append([])

In [ ]:
# listlengths = [len(filess_sum),len(filess_sum)+len(filess_doc),len(filess_sum)+len(filess_doc)+len(filess_sen),len(filess_sum)+len(filess_doc)+len(filess_sen)+len(filess_topi),len(filess_sum)+len(filess_doc)+len(filess_sen)+len(filess_topi)+len(filess_text),len(filess_sum)+len(filess_doc)+len(filess_sen)+len(filess_topi)+len(filess_text)+len(filess_relation),len(filess_sum)+len(filess_doc)+len(filess_sen)+len(filess_topi)+len(filess_text)+len(filess_relation)+len(filess_machine)]
listlengths = [409, 890, 1361, 1650, 2379, 2678, 3295]

In [ ]:
len(data_list_fin2)

292

In [ ]:
len(embeddings)

3270

In [ ]:
'''
We will compare the data points with the train data embeddings to see if any very different data points are present so just save it somewehre else so we will not feed
it to the KMwans model and so we will code more to make separate clusters
I put a threshold of 0.6 to check it, and it is a good threshold to keep.

Afterwards we will see the count of every sublist, if any count is less than 10 we will consider it to be in a new cluster
'''

for i in range(len(cosine_list)):
  for j in range(len(listlengths)):
    
    similarity = []
    count = 0
    if listlengths[j] == listlengths[0]:
      n = listlengths[j]
      similarity = cosine_similarity(embeddings_test[i].reshape(1,-1),embeddings.loc[0:n].values)
      for k in similarity:
        for l in k:
          if l > 0.6:
            count+=1
      cosine_list[i][j].append(count)
    
    else:
      similarity = cosine_similarity(embeddings_test[i].reshape(1,-1),embeddings.loc[n:listlengths[j]].values)
      n = listlengths[j]
      for k in similarity:
        for l in k:
          if l > 0.6:
            count+=1
      cosine_list[i][j].append(count)


  
        
      
  



  




#cosine_list[i][0].append(1)
# creating a list

total_list = []
# Iterate each element in list
# and add them in variale total
# for i in cosine_list:
#   total_list.append(sum(i))

cosine_list

[[[0], [0], [0], [0], [0], [0], [0]],
 [[6], [4], [3], [3], [6], [1], [2]],
 [[17], [15], [3], [17], [6], [3], [16]],
 [[3], [3], [6], [1], [4], [1], [14]],
 [[37], [43], [8], [17], [36], [31], [46]],
 [[0], [0], [0], [0], [0], [0], [0]],
 [[32], [32], [17], [30], [52], [28], [140]],
 [[58], [149], [47], [80], [100], [59], [136]],
 [[0], [0], [0], [0], [0], [0], [0]],
 [[69], [29], [4], [55], [28], [18], [24]],
 [[7], [10], [1], [2], [1], [1], [0]],
 [[4], [1], [3], [2], [2], [8], [10]],
 [[27], [72], [20], [34], [21], [36], [22]],
 [[0], [0], [0], [0], [0], [1], [0]],
 [[57], [83], [25], [107], [84], [63], [147]],
 [[19], [58], [21], [28], [62], [72], [136]],
 [[36], [24], [19], [9], [51], [30], [54]],
 [[6], [0], [1], [7], [0], [0], [6]],
 [[1], [3], [2], [3], [2], [0], [1]],
 [[40], [111], [20], [58], [53], [70], [72]],
 [[7], [33], [9], [19], [16], [23], [126]],
 [[1], [0], [0], [0], [0], [0], [1]],
 [[17], [36], [20], [22], [61], [87], [23]],
 [[28], [42], [18], [41], [67], [32], 

In [ ]:
data_list_fin3 = data_list_fin2

In [ ]:
len(cosine_list) == len(data_list_fin3)

True

In [ ]:
len(cosine_list)

292

In [ ]:
# data_list_fin2=[]
# for i in data_list2:
#   data_list_fin2.append(i[:2000])

In [ ]:
'''
Just to confirm, we will check the data points again to see if we did any duplication in the clustering data
If there is we will append it to a list to be given to the kmeans clustering algorithm
If there is not then we will make a new list and produce clusters out of it with the help Cosine similarity
'''

cluster_count = 7
check_list = []
check_list2 = []
check_list3 = []
mydata_list = []
topic_new_clusters = []
topic_old_clusters = []
count = 0
data_data = ['summarization','sentiment analysis','document classification','topic modeling','relation extraction','machine translation','text recognition','speech recognition']
cosine_count_list = []
cosine_count = 0
for i in range(len(cosine_list)):



  a = max(cosine_list[i])
  
  if a[0] < 10:
    print(i)
  

    for j in data_data:
      if j in data_list_fin3[i]:
        print(data_list_fin2[i])
        cosine_count_list.append(cosine_count)
        
        #data_list_fin3.pop(cosine_count)
        break
        
        
    cosine_count+=1
  else:
    cosine_count+=1
def Remove(duplicate):
    final_list = []
    for num in duplicate:
        if num not in final_list:
            final_list.append(num)
    return final_list
     
cosine_count_list = Remove(cosine_count_list)
cosine_count_list = Reverse(cosine_count_list)
for i in cosine_count_list:
 
  cosine_list.pop(i)
for i in cosine_count_list:
  mydata_list.append(data_list_fin3[i])
  topic_old_clusters.append(topic_list2[count])
  check_list3.append(embeddings_test[i])
  data_list_fin3.pop(i)

listt = []
for i in range(len(cosine_list)):
  a = max(cosine_list[i])
  if a[0] < 10:
    
    print(cluster_count)
    check_list.append(embeddings_test[count])
    check_list2.append(data_list_fin3[count])
    topic_new_clusters.append(topic_list2[count])
    listt.append(count)
     

    count+=1
    cluster_count+=1
  else:
    check_list3.append(embeddings_test[count])
    mydata_list.append(data_list_fin3[i])
    topic_old_clusters.append(topic_list2[count])
    print(cosine_list[i].index(a))
    count+=1

0
1
5
8
13
17
18
21
24
25
27
28
29
30
43
49
51
52
58
67
68
87
88
91
93
96
102
114
121
134
135
141
147
150
151
155
156
157
159
160
164
165
173
174
189
paper consider parallelization applications whose objective express maximize non monotone submodular function cardinality constraint main result algorithm whose approximation arbitrarily close log adaptive round size grind set exponential speedup parallel run time previously study algorithm constrain non monotone submodular maximization beyond provable guarantee algorithm perform well practice specifically experiment traffic monitor personalize data summarization applications show algorithm find solutions whose value competitive state art algorithms run exponentially fewer parallel iterations
194
196
198
199
201
210
212
213
parallel implementations stochastic gradient descent sgd receive significant research attention thank excellent scalability properties algorithm efficiency context train deep neural network fundamental barrier parallel

In [ ]:
len(check_list2)

79

In [ ]:
mydata = check_list2

In [ ]:
#This is the data which is to be used to cluster together with the help of cosine similarity NOT KMeans

mydata

['paper address graph match problem follow recent work cite zaslavskiy path vestner analyze generalize idea concave relaxations introduce concepts emph conditionally concave emph probably conditionally concave energies polytopes show encapsulate many instance graph match problem include match euclidean graph graph surface prove local minima probably conditionally concave energies general match polytopes doubly stochastic high probability extreme point match polytope permutations',
 'aim obtain interpretable expressive disentangle scene representation contain comprehensive structural textural information object previous scene representations learn neural network often uninterpretable limit single object lack knowledge work propose scene render network sdn address issue integrate disentangle representations semantics geometry appearance deep generative model scene encoder perform inverse graphics translate scene structure object wise representation decoder two components differentiable s

In [ ]:
#mydata_embeddings = model.encode(mydata,convert_to_tensor=False)


In [ ]:
mydata_embeddings = []
for i in range(len(mydata)):
  for j in range(len(data_list2)):
    if mydata[i] == data_list2[j]:
      #print(data_list2[j])
      mydata_embeddings.append(embeddings_test[j])
  




In [ ]:
'''
This is the code to produce clusters automatically with cosine similarity
This is kind of a big code and works really good. I have to work more in order to minimize the Time Complexity
But this code represents just 1 forward iteration
And so the end clusters are not that good
'''

list = []
topics_fin = []

length = len(mydata)
int = 0
for i in range(len(mydata_embeddings)):
  
  if len(mydata) > 0:
    sublist = []
    topics = []
    # mydata_embeddings = model.encode(mydata,convert_to_tensor=False)

  
  
    # remove = []
    # print('len')
    # print(len(mydata))
    # print('**')
    try:
      sublist.append(mydata[i])
      topics.append(topic_new_clusters[i])
    except:
      # print('*********')
      print(i)
    # mydata.pop(i)
    remove = []
    if int !=  length:
      for j in range(len(mydata_embeddings[1:])):
        similarity = cosine_similarity(mydata_embeddings[i].reshape(1,-1),mydata_embeddings[j+1].reshape(1,-1))
      
        if similarity > 0.5 and similarity < 0.99:
          #print(similarity)
          sublist.append(mydata[j+1])
          topics.append(topic_new_clusters[j+1])
          remove.append(j+1)
    int+= len(sublist)
    list.append(sublist)
    topics_fin.append(topics)
    remove = Reverse(remove)

    for i in remove:
      # print(i)
      mydata.pop(i)
      topic_new_clusters.pop(i)
    mydata_embeddings = model.encode(mydata,convert_to_tensor=False)
    print(len(mydata))
    print(len(topic_new_clusters))
    # print('len2')
    # print(len(mydata_embeddings))
    # print('**')





# sublist = []
# mydata_embeddings = model.encode(mydata,convert_to_tensor=False)
# for i in range(len(mydata_embeddings)):
 

#   remove = []
#   sublist.append(mydata[i])
#   remove.append(i)
#   for j in range(len(mydata_embeddings[1:])):
#     similarity = cosine_similarity(mydata_embeddings[i].reshape(1,-1),mydata_embeddings[j+1].reshape(1,-1))
  
#     if similarity > 0.5:
#       print(similarity)
#       sublist.append(mydata[j+1])
#       remove.append(j+1)

#   list.append(sublist)
#   remove = Reverse(remove)

#   for i in remove:
#     print(i)
#     mydata.pop(i)

#   break
  



# remove = Reverse(remove)

# for i in remove:
#   print(i)
#   mydata.pop(i)

    
  


77
77
62
62
60
60
58
58
54
54
38
38
38
38
36
36
36
36
36
36
34
34
34
34
34
34
34
34
34
34
34
34
33
33
31
31
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
31
30
30
32
30
30
33
30
30
34
30
30
35
30
30
36
30
30
37
30
30
38
30
30
39
30
30
40
30
30
41
30
30
42
30
30
43
30
30
44
30
30
45
30
30
46
30
30
47
30
30
48
30
30
49
30
30
50
30
30
51
30
30
52
30
30
53
30
30
54
30
30
55
30
30
56
30
30
57
30
30
58
30
30
59
30
30
60
30
30
61
30
30
62
30
30
63
30
30
64
30
30
65
30
30
66
30
30
67
30
30
68
30
30
69
30
30
70
30
30
71
30
30
72
30
30
73
30
30
74
30
30
75
30
30
76
30
30
77
30
30
78
30
30


In [ ]:
'''
Now this code uses the above list and performs 1 more reverse iteration so which means the end clusters produced from the above list are now much better
'''



import numpy as np
list2 = []
sim = []
empty_list = []
for i in Reverse(list):
  if len(i) > 0:
    list2.append(i)
list2_a = []
for i in list:
  if len(i) > 0:
    list2_a.append(i)
count = 0
list_made = []
for i in range(len(list2)):
  for j in range(len(list2[i])):
    print(list2[i][j])
    j2 = model.encode(list2[i][j], convert_to_tensor=False)
    sim = []
    embedd = []
    #instead of giving sentences as k, give embeddings as k to save a lot of time
    for k in range(len(list2_a)):
      #print(list2_a[k])
      embedd = []
      for m in range(len(list2_a[k])):
        for l in range(len(data_list2)):
          if list2_a[k][m] == data_list2[l]:
            embedd.append(embeddings_test[l])
      



      summ = []
      for z in embedd:
      #k2 = model.encode(list2_a[k], convert_to_tensor=False)
        similarity = cosine_similarity(j2.reshape(1,-1),z.reshape(1,-1))
        summ.append(similarity)
      sim.append(np.sum(summ)/len(list2_a[k]))
    
    maxi = max(sim)
    indexe = sim.index(maxi)
    sim.pop(indexe)
    maxi = max(sim)
    print(maxi)
    indexe = sim.index(maxi)
    if maxi > 0.47 and maxi < 0.98:
    
      #print(maxi)
      empty_list.append(list2[i][j])
      print(indexe)
      for ii in list2_a[indexe]:
        empty_list.append(ii)
      
      list2_a[indexe] = empty_list
      count+=1
      list2_a[-(i+1)][j] = 'nan'
      #print('.......')
      #print(empty_list)
      #print(list2_a[indexe])
      empty_list = []
      sim = []
  


    
  
      

      # for l in k:
      #   print(l)
      #   l2 = model.encode(l, convert_to_tensor=False)
      #   similarity = cosine_similarity(j2.reshape(1,-1),l2.reshape(1,-1))
      #   print(similarity)
    
        
      
  
        
      
 



infer intent observe behavior study extensively within frameworks bayesian inverse plan inverse reinforcement learn methods infer goal reward function best explain action observe agent typically human demonstrator another agent use infer intent predict imitate assist human user however central assumption inverse reinforcement learn demonstrator close optimal model suboptimal behavior exist typically assume suboptimal action result type random noise know cognitive bias like temporal inconsistency paper take alternative approach model suboptimal behavior result internal model misspecification reason user action might deviate near optimal action user incorrect set beliefs rule dynamics govern action affect environment insight demonstrate action may suboptimal real world may actually near optimal respect user internal model dynamics estimate internal beliefs observe behavior arrive new method infer intent demonstrate simulation user study participants approach enable accurately model human

In [ ]:
list2 = []
sim = []
empty_list = []
for i in Reverse(list):
  if len(i) > 0:
    list2.append(i)
list2_a = []
for i in list:
  if len(i) > 0:
    list2_a.append(i)
for k in range(len(list2_a)):
  k2 = model.encode(list2_a[k], convert_to_tensor=False)
  print(k2[0])
  break
  
      

[ 3.48688394e-01  1.14210509e-01  3.30486149e-01  2.54061431e-01
 -5.59634924e-01  3.54014218e-01  1.03870705e-01  1.42412037e-01
  3.18173200e-01  1.22078836e-01 -2.67956018e-01 -3.29017282e-01
  8.70834216e-02  5.19963622e-01  2.11012855e-01 -4.09339786e-01
  1.01074167e-01 -9.11884010e-02  1.31264076e-01 -2.11125836e-02
 -1.05277494e-01  6.46452531e-02 -2.22320825e-01 -3.70285124e-01
  1.92893520e-01  1.20432056e-01 -2.45764136e-01 -1.30696103e-01
 -2.31865406e-01 -3.11057836e-01  5.49843460e-02 -2.41370186e-01
 -5.63828982e-02  1.19837478e-01  2.35281765e-01 -2.51238555e-01
  1.32085755e-01 -3.65415066e-02 -1.52697459e-01  6.23982698e-02
  5.70226252e-01  5.24002850e-01 -6.64203707e-03  8.22785136e-04
 -1.81432277e-01 -3.10951382e-01  9.29651409e-03  3.67190510e-01
  1.93866283e-01  7.52349347e-02  3.16127390e-01  5.67721911e-02
 -2.61641234e-01  1.00542486e-01 -2.35177055e-01 -2.10447147e-01
 -1.05468489e-01  4.61082160e-01 -3.57637435e-01 -2.68475246e-02
 -1.19831190e-01 -4.93524

In [ ]:
len(k2[0])

768

In [ ]:
#print(list2_a[k])

for k in range(len(list2_a)):
  embedd = []
  for m in range(len(list2_a[k])):
    for l in range(len(data_list2)):
      if list2_a[k][m] == data_list2[l]:
        embedd.append(embeddings_test[l])
  print(embedd[0:])

  break
    


[array([ 3.48688573e-01,  1.14210613e-01,  3.30486178e-01,  2.54061282e-01,
       -5.59635103e-01,  3.54013711e-01,  1.03870757e-01,  1.42412066e-01,
        3.18173200e-01,  1.22079022e-01, -2.67955810e-01, -3.29017073e-01,
        8.70833695e-02,  5.19963562e-01,  2.11012900e-01, -4.09339905e-01,
        1.01073906e-01, -9.11883265e-02,  1.31264031e-01, -2.11125333e-02,
       -1.05277471e-01,  6.46450073e-02, -2.22320810e-01, -3.70284915e-01,
        1.92893609e-01,  1.20432071e-01, -2.45764345e-01, -1.30696133e-01,
       -2.31865272e-01, -3.11057925e-01,  5.49844950e-02, -2.41370395e-01,
       -5.63827232e-02,  1.19837396e-01,  2.35282019e-01, -2.51238585e-01,
        1.32085666e-01, -3.65415663e-02, -1.52697414e-01,  6.23982884e-02,
        5.70225954e-01,  5.24003029e-01, -6.64209668e-03,  8.22800852e-04,
       -1.81432307e-01, -3.10951442e-01,  9.29630548e-03,  3.67190570e-01,
        1.93866313e-01,  7.52348453e-02,  3.16127419e-01,  5.67723438e-02,
       -2.61641324e-01, 

In [ ]:
len(embedd[0:])

6

In [ ]:
list2_a = [['nan',
  'study problem policy policy evaluation oppe contrast prior work consider estimate individual policy value average policy value accurately draw inspiration recent work causal reason propose new finite sample generalization error bind value estimate mdp model use upper bind objective develop learn algorithm mdp model balance representation show approach yield substantially lower mse common synthetic benchmarks hiv treatment simulation domain',
  'nan',
  'adapt ideas underlie success deep learn continuous action domain present actor critic model free algorithm base deterministic policy gradient operate continuous action space use learn algorithm network architecture hyper parameters algorithm robustly solve simulate physics task include classic problems cartpole swing dexterous manipulation legged locomotion car drive algorithm able find policies whose performance competitive find plan algorithm full access dynamics domain derivatives demonstrate many task algorithm learn policies end end directly raw pixel input',
  'ideas enjoy large impact deep learn convolution problem involve pixels spatial representations common intuition hold convolutional neural network may appropriate paper show strike counterexample intuition via seemingly trivial coordinate transform problem simply require learn map coordinate cartesian space coordinate one hot pixel space although convolutional network would seem appropriate task show fail spectacularly demonstrate carefully analyze failure first toy problem point simple fix become obvious call solution coordconv work give convolution access input coordinate use extra coordinate channel without sacrifice computational parametric efficiency ordinary convolution coordconv allow network learn either complete translation invariance vary degrees translation dependence require end task coordconv solve coordinate transform problem perfect generalization time faster time fewer parameters convolution stark contrast raise question extent inability convolution persist insidiously inside task subtly hamper performance within complete answer question require investigation show preliminary evidence swap convolution coordconv improve model diverse set task use coordconv gin produce less mode collapse transform high level spatial latents pixels become easier learn faster cnn detection model train mnist detection show better iou use coordconv reinforcement learn domain agents play atari game benefit significantly use coordconv layer',
  'introduce new approach decomposable submodular function minimization dsfm exploit incidence relations incidence relations describe variables effectively influence component function properly utilize allow improve convergence rat dsfm solvers main result include precise parametrization dsfm problem base incidence relations development new scalable alternative projections parallel coordinate descent methods accompany rigorous analysis convergence rat',
  'propose structure adaptive variant state art stochastic variance reduce gradient algorithm katyusha regularize empirical risk minimization propose method able exploit intrinsic low dimensional structure solution sparsity low rank enforce non smooth regularization achieve even faster convergence rate provable algorithmic improvement do restart katyusha algorithm accord restrict strong convexity constants demonstrate effectiveness approach via numerical experiment',
  'give rigorous analysis statistical behavior gradients randomly initialize fully connect network relu activations result show empirical variance square entries input output jacobian exponential simple architecture dependent constant beta give sum reciprocals hide layer widths beta large gradients compute initialization vary wildly approach complement mean field theory analysis random network point view rigorously compute finite width corrections statistics gradients edge chaos',
  'present first accelerate randomize algorithm solve linear systems euclidean space one essential problem type matrix inversion problem particular algorithm specialize invert positive definite matrices way iterate approximate solutions generate algorithm positive definite matrices open way many applications field optimization machine learn application general theory develop first accelerate deterministic stochastic quasi newton update update lead provably aggressive approximations inverse hessian lead speed up classical non accelerate rule numerical experiment experiment empirical risk minimization show rule accelerate train machine learn model',
  'paper address graph match problem follow recent work cite zaslavskiy path vestner analyze generalize idea concave relaxations introduce concepts emph conditionally concave emph probably conditionally concave energies polytopes show encapsulate many instance graph match problem include match euclidean graph graph surface prove local minima probably conditionally concave energies general match polytopes doubly stochastic high probability extreme point match polytope permutations',
  'work introduce interactive structure learn framework unify many different interactive learn task present generalization query committee active learn algorithm set study consistency rate convergence theoretically empirically without noise',
  'convolutional neural network cnns recently achieve great success single image super resolution sisr however methods tend produce smooth output miss textural detail solve problems propose super resolution cliquenet srcliquenet reconstruct high resolution image better textural detail wavelet domain propose srcliquenet firstly extract set feature map low resolution image clique block group send set feature map clique sample module reconstruct image clique sample module consist four sub net predict high resolution wavelet coefficients four sub band since consider edge feature properties four sub band four sub net connect others learn coefficients four sub band jointly finally apply inverse discrete wavelet transform idwt output four sub net end clique sample module increase resolution reconstruct image extensive quantitative qualitative experiment benchmark datasets show method achieve superior performance state art methods'],
 ['nan',
  'nan',
  'nan',
  'nan',
  'policy gradient methods widely use control reinforcement learn particularly continuous action set host theoretically sound algorithms propose policy set due existence policy gradient theorem provide simplify form gradient policy learn however behaviour policy necessarily attempt learn follow optimal policy give task existence theorem elusive work solve open problem provide first policy policy gradient theorem key derivation use emphatic weight develop new actor critic algorithm call actor critic emphatic weight ace approximate simplify gradients provide theorem demonstrate simple counterexample previous policy policy gradient methods particularly offpac dpg converge wrong solution whereas ace find optimal solution',
  'nan',
  'large scale network model use neurons static nonlinearities produce analog output despite fact information process brain predominantly carry dynamic neurons produce discrete pulse call spike research spike base computation impede lack efficient supervise learn algorithm spike neural network present gradient descent method optimize spike network model introduce differentiable formulation spike dynamics derive exact gradient calculation demonstration train recurrent spike network two dynamic task one require optimize fast millisecond spike base interactions efficient encode information delay memory task extend duration second result show gradient descent approach indeed optimize network dynamics time scale individual spike well behavioral time scale conclusion method yield general purpose supervise learn algorithm spike neural network facilitate investigations spike base computations',
  'nan',
  'nan',
  'adapt ideas underlie success deep learn continuous action domain present actor critic model free algorithm base deterministic policy gradient operate continuous action space use learn algorithm network architecture hyper parameters algorithm robustly solve simulate physics task include classic problems cartpole swing dexterous manipulation legged locomotion car drive algorithm able find policies whose performance competitive find plan algorithm full access dynamics domain derivatives demonstrate many task algorithm learn policies end end directly raw pixel input',
  'accurate exposure key capture high quality photos computational photography especially mobile phone limit size camera modules inspire luminosity mask usually apply professional photographers paper develop novel algorithm learn local exposures deep reinforcement adversarial learn specific segment image sub image reflect variations dynamic range exposures accord raw low level feature base sub image local exposure sub image automatically learn virtue policy network sequentially reward learn globally design strike balance overall exposures aesthetic evaluation function approximate discriminator generative adversarial network reinforcement learn adversarial learn train collaboratively asynchronous deterministic policy gradient generative loss approximation simply algorithmic architecture also prove feasibility leverage discriminator value function employ local exposure retouch raw input image respectively thus deliver multiple retouch image different exposures fuse exposure blend extensive experiment verify algorithms superior state art methods term quantitative accuracy visual illustration',
  'nan',
  'nan',
  'nan',
  'nan',
  'study problem policy policy evaluation oppe contrast prior work consider estimate individual policy value average policy value accurately draw inspiration recent work causal reason propose new finite sample generalization error bind value estimate mdp model use upper bind objective develop learn algorithm mdp model balance representation show approach yield substantially lower mse common synthetic benchmarks hiv treatment simulation domain',
  'semantic scene completion predict volumetric occupancy object category scene help intelligent agents understand interact surround work propose disentangle framework sequentially carry semantic segmentation reprojection semantic scene completion three stage framework three advantage explicit semantic segmentation significantly boost performance flexible fusion ways sensor data bring good extensibility progress subtask promote holistic performance experimental result show regardless input single depth rgb framework generate high quality semantic scene completion outperform state art approach synthetic real datasets',
  'propose novel randomize first order optimization method sega sketch gradient method progressively throughout iterations build variance reduce estimate gradient random linear measurements sketch gradient provide iteration oracle iteration sega update current estimate gradient sketch project operation use information provide latest sketch subsequently use compute unbiased estimate true gradient random relaxation procedure unbiased estimate use perform gradient step unlike standard subspace descent methods coordinate descent sega use optimization problems non separable proximal term provide general convergence analysis prove linear convergence strongly convex objectives special case coordinate sketch sega enhance various techniques importance sample minibatching acceleration rate small constant factor identical best know rate coordinate descent',
  'despite remarkable advance image synthesis research exist work often fail manipulate image context large geometric transformations synthesize person image condition arbitrary pose one representative examples generation quality largely rely capability identify model arbitrary transformations different body part current generative model often build local convolutions overlook key challenge heavy occlusions different view dramatic appearance change distinct geometric change happen part cause arbitrary pose manipulations paper aim resolve challenge induce geometric variability spatial displacements via new soft gate warp generative adversarial network warp gin compose two stag first synthesize target part segmentation map give target pose depict region level spatial layouts guide image synthesis higher level structure constraints warp gin equip soft gate warp block learn feature level map render textures original image generate segmentation map warp gin capable control different transformation degrees give distinct target pose moreover propose warp block light weight flexible enough inject network human perceptual study quantitative evaluations demonstrate superiority warp gin significantly outperform exist methods two large datasets',
  'tremendous recent progress equilibrium find algorithms zero sum imperfect information extensive form game puzzle gap theory practice first order methods significantly better theoretical convergence rat counterfactual regret minimization cfr variant despite cfr variants favor practice experiment first order methods conduct small medium size game methods complicate implement set cfr variants enhance extensively decade perform well practice paper show particular first order method state art variant excessive gap technique instantiate dilate entropy distance function efficiently solve large real world problems competitively cfr variants show large endgames encounter libratus poker recently beat top human poker specialist professionals limit texas hold show experimental result variant excessive gap technique well prior version introduce numerically friendly implementation smooth best response computation associate first order methods extensive form game solve present knowledge first gpu implementation first order method extensive form game present comparisons several excessive gap technique cfr variants',
  'convolutional neural network cnns inherently subject invariable filter aggregate local input topological structure cause cnns allow manage data euclidean grid like structure image ones non euclidean graph structure traffic network broaden reach cnns develop structure aware convolution eliminate invariance yield unify mechanism deal euclidean non euclidean structure data technically filter structure aware convolution generalize univariate function capable aggregate local input diverse topological structure since infinite parameters require determine univariate function parameterize filter number learnable parameters context function approximation theory replace classical convolution cnns structure aware convolution structure aware convolutional neural network sacnns readily establish extensive experiment eleven datasets strongly evidence sacnns outperform current model various machine learn task include image classification cluster text categorization skeleton base action recognition molecular activity detection taxi flow prediction',
  'recent years supervise learn convolutional network cnns see huge adoption computer vision applications comparatively unsupervised learn cnns receive less attention work hope help bridge gap success cnns supervise learn unsupervised learn introduce class cnns call deep convolutional generative adversarial network dcgans certain architectural constraints demonstrate strong candidate unsupervised learn train various image datasets show convince evidence deep convolutional adversarial pair learn hierarchy representations object part scenes generator discriminator additionally use learn feature novel task demonstrate applicability general image representations',
  'recurrent neural network powerful tool understand model computation representation populations neurons continuous variable rate model network analyze apply extensively purpose however neurons fire action potentials discrete nature spike important feature neural circuit dynamics despite significant advance train recurrently connect spike neural network remain challenge present procedure train recurrently connect spike network generate dynamical pattern autonomously produce complex temporal output base integrate network input model physiological data procedure make use continuous variable network identify target train input spike model neurons surprisingly able construct spike network duplicate task perform continuous variable network relatively minor expansion number neurons approach provide novel view significance appropriate use fire rate model useful approach build model spike network use address important question representation computation neural systems'],
 ['present novel newton type method distribute optimization particularly well suit stochastic optimization learn problems quadratic objectives method enjoy linear rate convergence provably emph improve data size require essentially constant number iterations reasonable assumptions provide theoretical empirical evidence advantage method compare approach one shoot parameter average admm',
  'nan',
  'nan',
  'propose structure adaptive variant state art stochastic variance reduce gradient algorithm katyusha regularize empirical risk minimization propose method able exploit intrinsic low dimensional structure solution sparsity low rank enforce non smooth regularization achieve even faster convergence rate provable algorithmic improvement do restart katyusha algorithm accord restrict strong convexity constants demonstrate effectiveness approach via numerical experiment',
  'introduce new approach decomposable submodular function minimization dsfm exploit incidence relations incidence relations describe variables effectively influence component function properly utilize allow improve convergence rat dsfm solvers main result include precise parametrization dsfm problem base incidence relations development new scalable alternative projections parallel coordinate descent methods accompany rigorous analysis convergence rat'],
 ['describe iterative procedure optimize policies guarantee monotonic improvement make several approximations theoretically justify procedure develop practical algorithm call trust region policy optimization trpo algorithm similar natural policy gradient methods effective optimize large nonlinear policies neural network experiment demonstrate robust performance wide variety task learn simulate robotic swim hop walk gaits play atari game use image screen input despite approximations deviate theory trpo tend give monotonic improvement little tune hyperparameters',
  'nan',
  'nan',
  'attention network multimodal learn provide efficient way utilize give visual information selectively however computational cost learn attention distributions every pair multimodal input channel prohibitively expensive solve problem attention build two separate attention distributions modality neglect interaction multimodal input paper propose bilinear attention network ban find bilinear attention distributions utilize give vision language information seamlessly ban consider bilinear interactions among two group input channel low rank bilinear pool extract joint representations pair channel furthermore propose variant multimodal residual network exploit eight attention map ban efficiently quantitatively qualitatively evaluate model visual question answer vqa flickr entities datasets show ban significantly outperform previous methods achieve new state arts datasets',
  'recent work suggest enhance bloom filter use pre filter base apply machine learn determine function model data set bloom filter mean represent model learn bloom filter follow outcomes clarify guarantee associate structure show estimate size learn function must obtain order obtain improve performance provide simple method sandwich optimize learn bloom filter propose design analysis approach learn bloomier filter base model approach',
  'stochastic gradient descent sgd popular algorithm achieve state art performance variety machine learn task several researchers recently propose scheme parallelize sgd require performance destroy memory lock synchronization work aim show use novel theoretical analysis algorithms implementation sgd implement without lock present update scheme call hogwild allow processors access share memory possibility overwrite work show associate optimization problem sparse mean gradient update modify small part decision variable hogwild achieve nearly optimal rate convergence demonstrate experimentally hogwild outperform alternative scheme use lock order magnitude',
  'paper introduce versatile filter construct efficient convolutional neural network consider demand efficient deep learn techniques run cost effective hardware number methods develop learn compact neural network work aim slim filter different ways investigate small sparse binarized filter contrast treat filter additive perspective series secondary filter derive primary filter secondary filter inherit primary filter without occupy storage unfold computation could significantly enhance capability filter integrate information extract different receptive field besides spatial versatile filter additionally investigate versatile filter channel perspective new techniques general upgrade filter exist cnns experimental result benchmark datasets neural network demonstrate cnns construct versatile filter able achieve comparable accuracy original filter require less memory flop'],
 ['consider problem online learn linear contextual bandits set also strong individual fairness constraints govern unknown similarity metric constraints demand select similar action individuals approximately equal probability dhprz may odds optimize reward thus model settings profit social policy tension assume learn unknown mahalanobis similarity metric weak feedback identify fairness violations quantify extent intend represent interventions regulator know unfairness see nevertheless enunciate quantitative fairness metric individuals main result algorithm adversarial context set number fairness violations depend logarithmically obtain optimal sqrt regret bind best fair policy',
  'deep reinforcement learn successfully solve many challenge control task real world applicability limit inability ensure safety learn policies propose approach verifiable reinforcement learn train decision tree policies represent complex policies since nonparametric yet efficiently verify use exist techniques since highly structure challenge decision tree policies difficult train propose viper algorithm combine ideas model compression imitation learn learn decision tree policies guide dnn policy call oracle function show substantially outperform two baselines use viper learn provably robust decision tree policy variant atari pong symbolic state space learn decision tree policy toy game base pong provably never lose iii learn provably stable decision tree policy cart pole case decision tree policy achieve performance equal original dnn policy',
  'nan',
  'nan',
  'contextual bandit literature traditionally focus algorithms address exploration exploitation tradeoff particular greedy algorithms exploit current estimate without exploration may sub optimal general however exploration free greedy algorithms desirable practical settings exploration may costly unethical clinical trials surprisingly find simple greedy algorithm rate optimal achieve asymptotically optimal regret sufficient randomness observe contexts covariates prove always case two arm bandit general class context distributions satisfy condition term covariate diversity furthermore even absent condition show greedy algorithm rate optimal positive probability thus standard bandit algorithms may unnecessarily explore motivate result introduce greedy first new algorithm use observe contexts reward determine whether follow greedy algorithm explore prove algorithm rate optimal without additional assumptions context distribution number arm extensive simulations demonstrate greedy first successfully reduce exploration outperform exist exploration base contextual bandit algorithms thompson sample upper confidence bind ucb',
  'consider problem online learn linear contextual bandits set also strong individual fairness constraints govern unknown similarity metric constraints demand select similar action individuals approximately equal probability dhprz may odds optimize reward thus model settings profit social policy tension assume learn unknown mahalanobis similarity metric weak feedback identify fairness violations quantify extent intend represent interventions regulator know unfairness see nevertheless enunciate quantitative fairness metric individuals main result algorithm adversarial context set number fairness violations depend logarithmically obtain optimal sqrt regret bind best fair policy',
  'humans routinely retrace path novel environment forward backwards despite uncertainty motion paper present approach give demonstration path first network generate path equip second network observe world decide act order retrace path noisy actuation change environment two network optimize end end train time evaluate method two realistic simulators perform path follow forward backwards experiment show approach outperform classical approach solve task well number baselines',
  'humans make repeat choices among options imperfectly know reward outcomes important problem psychology neuroscience often study use multi arm bandits also frequently study machine learn present data human stationary bandit experiment vary average abundance variability reward availability mean variance reward rate distributions surprisingly find subject significantly underestimate prior mean reward rat base self report end game reward expectation non choose arm previously human learn bandit task find well capture bayesian ideal learn model dynamic belief model dbm albeit incorrect generative assumption temporal structure humans assume reward rat change time even though actually fix find pessimism bias bandit task well capture prior mean dbm fit human choices poorly capture prior mean fix belief model fbm alternative bayesian model correctly assume reward rat constants pessimism bias also incompletely capture simple reinforcement learn model commonly use neuroscience psychology term fit initial value seem sub optimal thus mysterious humans underestimate prior reward expectation simulations show underestimate prior mean help maximize long term gain observer assume volatility reward rat stable utilize softmax decision policy instead optimal one obtainable dynamic program raise intrigue possibility brain underestimate reward rat compensate incorrect non stationarity assumption generative model simplify decision policy'],
 ['nan',
  'nan',
  'algorithmically construct multi output gaussian process priors satisfy linear differential equations approach attempt parametrize solutions equations use bner base successful push forward gaussian process along paramerization desire prior consider several examples physics geomathmatics control among full inhomogeneous system maxwell equations bring together stochastic learn computeralgebra novel way combine noisy observations precise algebraic computations',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'note consider normalize gradient descent ngd natural modification classical gradient descent optimization problems serious shortcoming non convex problems may take arbitrarily long escape neighborhood saddle point issue make convergence arbitrarily slow particularly high dimensional non convex problems relative number saddle point often large paper focus continuous time descent show contrary standard ngd escape saddle point quickly particular show ngd almost never converge saddle point time require ngd escape ball radius saddle point sqrt kappa kappa condition number hessian application result global convergence time bind establish ngd mild assumptions',
  'nan',
  'present shapenet richly annotate large scale repository shape represent cad model object shapenet contain model multitude semantic categories organize wordnet taxonomy collection datasets provide many semantic annotations model consistent rigid alignments part bilateral symmetry plan physical size keywords well plan annotations annotations make available public web base interface enable data visualization object attribute promote data drive geometric analysis provide large scale quantitative benchmark research computer graphics vision time technical report shapenet index model model classify categories wordnet synsets report describe shapenet effort whole provide detail currently available datasets summarize future plan',
  'nan',
  'wide adoption dnns give birth unrelenting compute requirements force datacenter operators adopt domain specific accelerators train accelerators typically employ densely pack full precision float point arithmetic maximize performance per area ongoing research efforts seek increase performance density replace float point fix point arithmetic however significant roadblock attempt fix point narrow dynamic range insufficient dnn train convergence identify block float point bfp promise alternative representation since exhibit wide dynamic range enable majority dnn operations perform fix point logic unfortunately bfp alone introduce several limitations preclude direct applicability work introduce hbfp hybrid bfp approach perform dot products bfp operations float point hbfp deliver best worlds high accuracy float point superior hardware density fix point wide variety model show hbfp match float point accuracy enable hardware implementations deliver higher throughput',
  'nan',
  'nan'],
 ['machine learn model vulnerable adversarial examples small change image cause computer vision model make mistake identify school bus ostrich however still open question whether humans prone similar mistake address question leverage recent techniques transfer adversarial examples computer vision model know parameters architecture model unknown parameters architecture match initial process human visual system find adversarial examples strongly transfer across computer vision model influence classifications make time limit human observers'],
 ['paper address graph match problem follow recent work cite zaslavskiy path vestner analyze generalize idea concave relaxations introduce concepts emph conditionally concave emph probably conditionally concave energies polytopes show encapsulate many instance graph match problem include match euclidean graph graph surface prove local minima probably conditionally concave energies general match polytopes doubly stochastic high probability extreme point match polytope permutations',
  'ensembles randomize decision tree usually refer random forest widely use classification regression task machine learn statistics random forest achieve competitive predictive performance computationally efficient train test make excellent candidates real world prediction task popular random forest variants breiman random forest extremely randomize tree operate batch train data online methods greater demand exist online random forest however require train data batch counterpart achieve comparable predictive performance work use mondrian process roy teh construct ensembles random decision tree call mondrian forest mondrian forest grow incremental online fashion remarkably distribution online mondrian forest batch mondrian forest mondrian forest achieve competitive predictive performance comparable exist online random forest periodically train batch random forest order magnitude faster thus represent better computation accuracy tradeoff',
  'nan',
  'nan',
  'deep reinforcement learn successfully solve many challenge control task real world applicability limit inability ensure safety learn policies propose approach verifiable reinforcement learn train decision tree policies represent complex policies since nonparametric yet efficiently verify use exist techniques since highly structure challenge decision tree policies difficult train propose viper algorithm combine ideas model compression imitation learn learn decision tree policies guide dnn policy call oracle function show substantially outperform two baselines use viper learn provably robust decision tree policy variant atari pong symbolic state space learn decision tree policy toy game base pong provably never lose iii learn provably stable decision tree policy cart pole case decision tree policy achieve performance equal original dnn policy'],
 ['paper try organize machine teach coherent set ideas idea present vary along dimension collection dimension form problem space machine teach exist teach problems characterize space hope organization allow gain deeper understand individual teach problems discover connections among identify gap field'],
 ['goal orient dialog give attention due numerous applications artificial intelligence goal orient dialogue task occur questioner ask action orient question answerer respond intent let questioner know correct action take ask adequate question deep learn reinforcement learn recently apply however approach struggle find competent recurrent neural questioner owe complexity learn series sentence motivate theory mind propose answerer questioner mind aqm novel information theoretic algorithm goal orient dialog aqm questioner ask infer base approximate probabilistic model answerer questioner figure answerer intention via select plausible question explicitly calculate information gain candidate intentions possible answer question test framework two goal orient visual dialog task mnist count dialog guesswhat experiment aqm outperform comparative algorithms large margin'],
 ['nan',
  'bilinear model show achieve impressive performance wide range visual task semantic segmentation fine grain recognition face recognition however bilinear feature high dimensional typically order hundreds thousands million make impractical subsequent analysis propose two compact bilinear representations discriminative power full bilinear representation thousand dimension compact representations allow back propagation classification errors enable end end optimization visual recognition system compact bilinear representations derive novel kernelized analysis bilinear pool provide insights discriminative power bilinear pool platform research compact pool methods experimentation illustrate utility propose representations image classification shoot learn across several datasets',
  'nan'],
 ['aim obtain interpretable expressive disentangle scene representation contain comprehensive structural textural information object previous scene representations learn neural network often uninterpretable limit single object lack knowledge work propose scene render network sdn address issue integrate disentangle representations semantics geometry appearance deep generative model scene encoder perform inverse graphics translate scene structure object wise representation decoder two components differentiable shape renderer neural texture generator disentanglement semantics geometry appearance support aware scene manipulation rotate move object freely keep consistent shape texture change object appearance without affect shape experiment demonstrate edit scheme base sdn superior counterpart',
  'accurately answer question give image require combine observations general knowledge effortless humans reason general knowledge remain algorithmic challenge advance research direction novel reason correct answer jointly consider entities show challenge fvqa dataset lead improvement accuracy around compare state art',
  'reason play essential role visual question answer vqa multi step dynamic reason often necessary answer complex question example question place next bus right picture talk compound object bus right generate relation'],
 ['hash one popular powerful approximate nearest neighbor search techniques large scale image retrieval traditional hash methods first represent image shelf visual feature produce hash cod separate stage however shelf visual feature may optimally compatible hash code learn procedure may result sub optimal hash cod recently deep hash methods propose simultaneously learn image feature hash cod use deep neural network show superior performance traditional hash methods deep hash methods give supervise information form pairwise label triplet label current state art deep hash method dpsh cite feature base pairwise label perform image feature learn hash code learn simultaneously maximize likelihood pairwise similarities inspire dpsh cite feature propose triplet label base deep hash method aim maximize likelihood give triplet label experimental result show method outperform baselines cifar nus wide datasets include state art method dpsh cite feature previous triplet label base deep hash methods'],
 ['learn decision tree data difficult optimization problem widespread algorithm practice date base greedy growth tree structure recursively split nod possibly prune back final tree parameters decision function internal node approximately estimate minimize impurity measure give algorithm give input tree structure parameter value nod produce new tree smaller structure new parameter value provably lower leave unchanged misclassification error apply axis align oblique tree experiment show consistently outperform various algorithms highly scalable large datasets tree algorithm handle sparsity penalty learn sparse oblique tree structure subset original tree nonzero parameters combine best axis align oblique tree flexibility model correlate data low generalization error fast inference interpretable nod involve feature decision',
  'paper compare different type recurrent units recurrent neural network rnns especially focus sophisticate units implement gate mechanism long short term memory lstm unit recently propose gate recurrent unit gru evaluate recurrent units task polyphonic music model speech signal model experiment reveal advance recurrent units indeed better traditional recurrent units tanh units also find gru comparable lstm'],
 ['tremendous recent progress equilibrium find algorithms zero sum imperfect information extensive form game puzzle gap theory practice first order methods significantly better theoretical convergence rat counterfactual regret minimization cfr variant despite cfr variants favor practice experiment first order methods conduct small medium size game methods complicate implement set cfr variants enhance extensively decade perform well practice paper show particular first order method state art variant excessive gap technique instantiate dilate entropy distance function efficiently solve large real world problems competitively cfr variants show large endgames encounter libratus poker recently beat top human poker specialist professionals limit texas hold show experimental result variant excessive gap technique well prior version introduce numerically friendly implementation smooth best response computation associate first order methods extensive form game solve present knowledge first gpu implementation first order method extensive form game present comparisons several excessive gap technique cfr variants',
  'contextual bandit literature traditionally focus algorithms address exploration exploitation tradeoff particular greedy algorithms exploit current estimate without exploration may sub optimal general however exploration free greedy algorithms desirable practical settings exploration may costly unethical clinical trials surprisingly find simple greedy algorithm rate optimal achieve asymptotically optimal regret sufficient randomness observe contexts covariates prove always case two arm bandit general class context distributions satisfy condition term covariate diversity furthermore even absent condition show greedy algorithm rate optimal positive probability thus standard bandit algorithms may unnecessarily explore motivate result introduce greedy first new algorithm use observe contexts reward determine whether follow greedy algorithm explore prove algorithm rate optimal without additional assumptions context distribution number arm extensive simulations demonstrate greedy first successfully reduce exploration outperform exist exploration base contextual bandit algorithms thompson sample upper confidence bind ucb',
  'societies often rely human experts take wide variety decisions affect members jail release decisions take judge stop frisk decisions take police officer accept reject decisions take academics context decision take expert typically choose uniformly random pool experts however decisions may imperfect due limit experience implicit bias faulty probabilistic reason improve accuracy fairness overall decision make process optimize assignment experts decisions'],
 ['draw attention important yet largely overlook aspect evaluate fairness automate decision make systems namely risk welfare considerations propose family measure correspond long establish formulations cardinal social welfare economics justify rawlsian conception fairness behind veil ignorance convex formulation welfare base measure fairness allow integrate constraint convex loss minimization pipeline empirical analysis reveal interest trade off proposal prediction accuracy group discrimination dwork notion individual fairness furthermore perhaps importantly work provide heuristic justification empirical evidence suggest lower bind measure often lead bound inequality algorithmic outcomes hence present first computationally feasible mechanism bound individual level inequality'],
 ['technical challenge deep learn recognize target class without see data zero shoot learn leverage semantic representations attribute class prototypes bridge source target class exist standard zero shoot learn methods may prone overfitting see data source class blind semantic representations target class paper study generalize zero shoot learn assume accessible target class unseen data train prediction unseen data make search source target class propose novel deep calibration network dcn approach towards generalize zero shoot learn paradigm enable simultaneous calibration deep network confidence source class uncertainty target class approach map visual feature image semantic representations class prototypes common embed space compatibility see data source target class maximize show superior accuracy approach state art benchmark datasets generalize zero shoot learn include awa cub sun apy',
  'paper propose conceptually simple general framework call metagan shoot learn problems state art shoot classification model integrate metagan principled straightforward way introduce adversarial generator condition task augment vanilla shoot classification model ability discriminate real fake data argue gin base approach help shoot classifiers learn sharper decision boundary could generalize better show metagan framework extend supervise shoot learn model naturally cope unsupervised data different previous work semi supervise shoot learn algorithms deal semi supervision sample level task level give theoretical justifications strength metagan validate effectiveness metagan challenge shoot image classification benchmarks'],
 ['nan',
  'nan',
  'recurrent network spike neurons rsnns underlie astound compute learn capabilities brain compute learn capabilities rsnn model remain poor least comparison anns address two possible reason one rsnns brain randomly connect design accord simple rule start learn tabula rasa network rather rsnns brain optimize task evolution development prior experience detail optimization process largely unknown functional contribution approximate powerful optimization methods backpropagation time bptt'],
 ['suppose design matrix linear regression problem give response point hide unless explicitly request goal sample small number responses produce weight vector whose sum square loss point epsilon time minimum small jointly sample diverse subsets point crucial one method call volume sample unique desirable property weight vector produce unbiased estimate optimum therefore natural ask method offer optimal unbiased estimate term number responses need achieve epsilon loss approximation',
  'consider problem learn optimal reserve price repeat auction non myopic bidders may bid strategically order gain future round even single round auction truthful previous algorithms empirical price provide non trivial regret round set general introduce algorithms obtain small regret non myopic bidders either market large bidder appear constant fraction round bidders impatient discount future utility factor mildly bound away one approach carefully control information reveal bidder build techniques differentially private online learn well recent line work jointly differentially private algorithms',
  'duplicate removal critical step accomplish reasonable amount predictions prevalent proposal base object detection frameworks albeit simple effective previous algorithms utilize greedy process without make sufficient use properties input data work design new two stage framework effectively select appropriate proposal candidate object first stage suppress easy negative object proposals second stage select true positives reduce proposal set two stag share network structure encoder decoder form recurrent neural network rnn global attention context gate encoder scan proposal candidates sequential manner capture global context information feed decoder extract optimal proposals extensive experiment propose method outperform alternatives large margin'],
 ['significant interest able predict crimes happen example aid efficient task police protective measure aim model temporal spatial dependencies often exhibit violent crimes order make predictions temporal variation crimes typically follow pattern familiar time series analysis spatial pattern irregular vary smoothly across area instead find spatially disjoint regions exhibit correlate crime pattern indeterminate inter region correlation structure along low count discrete nature count serious crimes motivate propose forecast tool particular propose model crime count region use integer value first order autoregressive process take bayesian nonparametric approach flexibly discover cluster region specific time series describe account covariates within framework approach adjust seasonality demonstrate approach analysis weekly report violent crimes washington forecast outperform standard methods additionally provide useful tool prediction intervals',
  'decision tree random forest well establish model offer good predictive performance also provide rich feature importance information practitioners often employ variable importance methods rely impurity base information methods remain poorly characterize theoretical perspective provide novel insights performance methods derive finite sample performance guarantee high dimensional set various model assumptions demonstrate effectiveness impurity base methods via extensive set simulations',
  'several large scale deployments differential privacy use collect statistical information users however deployments periodically recollect data recompute statistics use algorithms design single use result systems provide meaningful privacy guarantee long time scale moreover exist techniques mitigate effect apply local model differential privacy systems use'],
 ['propose novel flexible anchor mechanism name metaanchor object detection frameworks unlike many previous detectors model anchor via predefined manner metaanchor anchor function could dynamically generate arbitrary customize prior box take advantage weight prediction metaanchor able work anchor base object detection systems retinanet compare predefined anchor scheme empirically find metaanchor robust anchor settings bound box distributions addition also show potential transfer task experiment coco detection task show metaanchor consistently outperform counterparts various scenarios'],
 ['nan',
  'accurately answer question give image require combine observations general knowledge effortless humans reason general knowledge remain algorithmic challenge advance research direction novel reason correct answer jointly consider entities show challenge fvqa dataset lead improvement accuracy around compare state art'],
 ['despite remarkable advance image synthesis research exist work often fail manipulate image context large geometric transformations synthesize person image condition arbitrary pose one representative examples generation quality largely rely capability identify model arbitrary transformations different body part current generative model often build local convolutions overlook key challenge heavy occlusions different view dramatic appearance change distinct geometric change happen part cause arbitrary pose manipulations paper aim resolve challenge induce geometric variability spatial displacements via new soft gate warp generative adversarial network warp gin compose two stag first synthesize target part segmentation map give target pose depict region level spatial layouts guide image synthesis higher level structure constraints warp gin equip soft gate warp block learn feature level map render textures original image generate segmentation map warp gin capable control different transformation degrees give distinct target pose moreover propose warp block light weight flexible enough inject network human perceptual study quantitative evaluations demonstrate superiority warp gin significantly outperform exist methods two large datasets',
  'generative adversarial network gans technique learn generative model complex data distributions sample despite remarkable advance generate realistic image major shortcoming gans fact tend produce sample little diversity even train diverse datasets phenomenon know mode collapse focus much recent work study principled approach handle mode collapse call pack main idea modify discriminator make decisions base multiple sample class either real artificially generate draw analysis tool binary hypothesis test particular seminal result blackwell prove fundamental connection pack mode collapse show pack naturally penalize generators mode collapse thereby favor generator distributions less mode collapse train process numerical experiment benchmark datasets suggest pack provide significant improvements',
  'nan'],
 ['recent years supervise learn convolutional network cnns see huge adoption computer vision applications comparatively unsupervised learn cnns receive less attention work hope help bridge gap success cnns supervise learn unsupervised learn introduce class cnns call deep convolutional generative adversarial network dcgans certain architectural constraints demonstrate strong candidate unsupervised learn train various image datasets show convince evidence deep convolutional adversarial pair learn hierarchy representations object part scenes generator discriminator additionally use learn feature novel task demonstrate applicability general image representations',
  'accurate exposure key capture high quality photos computational photography especially mobile phone limit size camera modules inspire luminosity mask usually apply professional photographers paper develop novel algorithm learn local exposures deep reinforcement adversarial learn specific segment image sub image reflect variations dynamic range exposures accord raw low level feature base sub image local exposure sub image automatically learn virtue policy network sequentially reward learn globally design strike balance overall exposures aesthetic evaluation function approximate discriminator generative adversarial network reinforcement learn adversarial learn train collaboratively asynchronous deterministic policy gradient generative loss approximation simply algorithmic architecture also prove feasibility leverage discriminator value function employ local exposure retouch raw input image respectively thus deliver multiple retouch image different exposures fuse exposure blend extensive experiment verify algorithms superior state art methods term quantitative accuracy visual illustration',
  'people belong multiple communities word belong multiple topics book cover multiple genres overlap cluster commonplace many exist overlap cluster methods model person word book non negative weight combination exemplars belong solely one community small noise geometrically person point cone whose corner exemplars basic form encompass widely use mix membership stochastic blockmodel network degree correct variants well topic model lda show simple one class svm yield provably consistent parameter inference model scale large datasets experimental result several simulate real datasets show algorithm call svm cone accurate scalable'],
 ['convolutional neural network cnns recently achieve great success single image super resolution sisr however methods tend produce smooth output miss textural detail solve problems propose super resolution cliquenet srcliquenet reconstruct high resolution image better textural detail wavelet domain propose srcliquenet firstly extract set feature map low resolution image clique block group send set feature map clique sample module reconstruct image clique sample module consist four sub net predict high resolution wavelet coefficients four sub band since consider edge feature properties four sub band four sub net connect others learn coefficients four sub band jointly finally apply inverse discrete wavelet transform idwt output four sub net end clique sample module increase resolution reconstruct image extensive quantitative qualitative experiment benchmark datasets show method achieve superior performance state art methods',
  'paper introduce versatile filter construct efficient convolutional neural network consider demand efficient deep learn techniques run cost effective hardware number methods develop learn compact neural network work aim slim filter different ways investigate small sparse binarized filter contrast treat filter additive perspective series secondary filter derive primary filter secondary filter inherit primary filter without occupy storage unfold computation could significantly enhance capability filter integrate information extract different receptive field besides spatial versatile filter additionally investigate versatile filter channel perspective new techniques general upgrade filter exist cnns experimental result benchmark datasets neural network demonstrate cnns construct versatile filter able achieve comparable accuracy original filter require less memory flop',
  'humans routinely retrace path novel environment forward backwards despite uncertainty motion paper present approach give demonstration path first network generate path equip second network observe world decide act order retrace path noisy actuation change environment two network optimize end end train time evaluate method two realistic simulators perform path follow forward backwards experiment show approach outperform classical approach solve task well number baselines',
  'generative recurrent neural network quickly train unsupervised manner model popular reinforcement learn environments compress spatio temporal representations world model extract feature feed compact simple policies train evolution achieve state art result various environments also train agent entirely inside environment generate internal world model transfer policy back actual environment interactive version paper available https worldmodels github',
  'introduce approach convert mono audio record video camera spatial audio representation distribution sound full view sphere spatial audio important component immersive video view spatial audio microphones still rare current video production system consist end end trainable neural network separate individual sound source localize view sphere condition multi modal analysis audio video frame introduce several datasets include one film one collect wild youtube consist videos upload spatial audio train grind truth spatial audio serve self supervision mix mono track form input network use approach show possible infer spatial localization sound base synchronize video mono audio track'],
 ['propose simple yet effective approach spatiotemporal feature learn use deep dimensional convolutional network convnets train large scale supervise video dataset find three fold convnets suitable spatiotemporal feature learn compare convnets homogeneous architecture small convolution kernels layer among best perform architectures convnets learn feature namely convolutional simple linear classifier outperform state art methods different benchmarks comparable current best methods benchmarks addition feature compact achieve accuracy ucf dataset dimension also efficient compute due fast inference convnets finally conceptually simple easy train use',
  'softmax output activation function model categorical probability distributions many applications deep learn however recent study reveal softmax bottleneck representational capacity neural network language model softmax bottleneck paper propose output activation function break softmax bottleneck without additional parameters analyze softmax bottleneck perspective output set log softmax identify cause softmax bottleneck basis analysis propose sigsoftmax compose multiplication exponential function sigmoid function sigsoftmax break softmax bottleneck experiment language model demonstrate sigsoftmax mixture sigsoftmax outperform softmax mixture softmax respectively'],
 ['real world applications education effective teacher adaptively choose next example teach base learner current state however exist work algorithmic machine teach focus batch set adaptivity play role paper study case teach consistent version space learners interactive set time step teacher provide example learner perform update teacher observe learner new state highlight adaptivity speed teach process consider exist model version space learners worst case model learner pick next hypothesis randomly version space preference base model learner pick hypothesis accord global preference inspire human teach propose new model learner pick hypotheses accord local preference define current hypothesis show model exhibit several desirable properties adaptivity play key role learner transition hypotheses smooth interpretable develop adaptive teach algorithms demonstrate result via simulation user study'],
 ['nan'],
 ['nan'],
 ['nan']]

In [ ]:
list2_a2 = []
length = len(list2_a)
for i in range(length):
  list2_a2.append([])
for i in range(len(list2_a)):
  for j in range(len(list2_a[i])):
    if list2_a[i][j] == 'nan':
      pass
    
    else:
      list2_a2[i].append(list2_a[i][j])
list2_a22 = []
for i in list2_a2:
  if len(i) == 0:
    pass
  else:
    list2_a22.append(i)

In [ ]:
list2_a22

[['study problem policy policy evaluation oppe contrast prior work consider estimate individual policy value average policy value accurately draw inspiration recent work causal reason propose new finite sample generalization error bind value estimate mdp model use upper bind objective develop learn algorithm mdp model balance representation show approach yield substantially lower mse common synthetic benchmarks hiv treatment simulation domain',
  'adapt ideas underlie success deep learn continuous action domain present actor critic model free algorithm base deterministic policy gradient operate continuous action space use learn algorithm network architecture hyper parameters algorithm robustly solve simulate physics task include classic problems cartpole swing dexterous manipulation legged locomotion car drive algorithm able find policies whose performance competitive find plan algorithm full access dynamics domain derivatives demonstrate many task algorithm learn policies end end dire

In [ ]:
#Just to see how many data points changed their position 

count = []
for i in list2:
  count.append(len(i))
print(Reverse(count))
count = []
for i in list2_a:
  count.append(len(i))
print(count)


[3, 16, 3, 3, 5, 17, 1, 3, 1, 1, 3, 1, 1, 1, 1, 1, 2, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[9, 28, 8, 8, 12, 19, 1, 4, 1, 3, 3, 3, 1, 2, 3, 2, 2, 4, 3, 2, 1, 3, 2, 1, 1, 1, 4, 1, 6, 1]


In [ ]:
#Predicting the training data with KMeans

from sklearn.cluster import KMeans
num_clusters = 7
# Define kmeans model
clustering_model = KMeans(n_clusters=num_clusters)
# Fit the embedding with kmeans clustering.
clustering_model.fit(embeddings)
# Get the cluster id assigned to each news headline.
clusterr = clustering_model.get_params
cluster_assignment = clustering_model.labels_


In [ ]:
predicted = clustering_model.predict(check_list3)



In [ ]:
predicted

array([3, 5, 1, 1, 3, 1, 3, 1, 5, 1, 3, 1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 6,
       2, 1, 3, 1, 1, 1, 1, 3, 1, 3, 1, 1, 3, 3, 1, 1, 2, 3, 1, 3, 1, 1,
       1, 3, 3, 2, 3, 3, 1, 3, 0, 1, 3, 6, 3, 1, 1, 1, 3, 1, 2, 1, 3, 5,
       3, 3, 3, 3, 5, 3, 3, 3, 1, 6, 1, 1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 5,
       5, 3, 0, 2, 1, 0, 1, 1, 1, 5, 1, 1, 3, 2, 2, 1, 1, 3, 1, 3, 1, 1,
       3, 1, 1, 1, 3, 1, 3, 3, 1, 1, 3, 1, 1, 1, 1, 3, 5, 1, 1, 1, 1, 1,
       1, 3, 3, 5, 3, 3, 1, 1, 1, 3, 3, 1, 5, 1, 1, 5, 1, 3, 1, 1, 5, 3,
       3, 3, 1, 1, 1, 3, 5, 1, 3, 4, 3, 5, 1, 3, 1, 1, 5, 1, 6, 3, 3, 1,
       1, 5, 1, 5, 3, 2, 3, 3, 5, 3, 3, 1, 1, 1, 1, 3, 3, 1, 3, 1, 3, 1,
       1, 1, 2, 5, 1, 1, 1, 3, 2, 3, 3, 3, 3, 3, 3], dtype=int32)

In [ ]:
# As I said before there are 2 lists ( 1 which will be clustered from KMeans and other from cosine similarity)
# We clustered one list with cosine
# Now we will cluster the other list with KMeans (trained on training data)

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(predicted):
    clustered_sentences[cluster_id].append(mydata_list[sentence_id])
for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

Cluster  1
['great progress make recently automatic image manipulation limit object centric image like face structure scene datasets work take step towards general scene level image edit develop automatic interaction free object removal model model learn find remove object general scene image use image level label unpaired data generative adversarial network gin framework achieve two key contributions two stage editor architecture consist mask generator image painter operate remove object novel gin base prior mask generator allow flexibly incorporate knowledge object shape experimentally show two datasets method effectively remove wide variety object use weak supervision', 'exist methods interactive image retrieval demonstrate merit integrate user feedback improve retrieval result however current systems rely restrict form user feedback binary relevance responses feedback base fix set relative attribute limit impact paper introduce new approach interactive image search enable users pro

In [ ]:
final_list = []
for i in clustered_sentences:
  if len(i) > 0:
    final_list.append(i)
for i in list2_a:
  if len(i) > 0:
    final_list.append(i)
len(final_list)


33

In [ ]:
# Final list containing all the CLUSTERS

final_list

[['great progress make recently automatic image manipulation limit object centric image like face structure scene datasets work take step towards general scene level image edit develop automatic interaction free object removal model model learn find remove object general scene image use image level label unpaired data generative adversarial network gin framework achieve two key contributions two stage editor architecture consist mask generator image painter operate remove object novel gin base prior mask generator allow flexibly incorporate knowledge object shape experimentally show two datasets method effectively remove wide variety object use weak supervision',
  'exist methods interactive image retrieval demonstrate merit integrate user feedback improve retrieval result however current systems rely restrict form user feedback binary relevance responses feedback base fix set relative attribute limit impact paper introduce new approach interactive image search enable users provide fee

In [ ]:
# Just appending all the sentences of a cluster into 1 string

appended_final_list = []
for i in final_list:
    result= ''
    for element in i:
        result += str(element)
    appended_final_list.append(result)
    

In [ ]:
appended_final_list = ['although various machine learn base predictors develop estimate protein protein interactions performances vary dataset species affect two primary aspects choice learn algorithm representation protein pair improve performance predict protein protein interactions exploit synergy multiple learn algorithms utilize expressiveness different protein pair feature develop stack generalization scheme integrate five learn algorithms also design three type protein pair feature base physicochemical properties amino acids gene ontology annotations interaction network topologies test publish datasets collect eight species propose approach achieve significantly higher comparable overall performance compare seven competitive predictors introduce ensemble learn approach ppi prediction integrate multiple learn algorithms different protein pair representations extensive comparisons state art prediction tool demonstrate feasibility superiority propose methodbackground increase amount structure data publish use principles link data mean likely find resources web data describe real life concepts however discover resources relate give resource still open research area thesis study recommender systems use link data source generate recommendations exploit large amount available resources relationships among aim main objective study propose recommendation tech nique resources consider semantic relationships concepts link data specific objectives define semantic relationships derive resources take account knowledge find link data datasets determine semantic similarity measure base semantic relationships derive resources iii propose algorithm dynami cally generate automatic rank resources accord define similarity measure methodology base recommendations project management institute integral model engineer professionals universidad del cauca first one manage project second one develop experimental prototype accordingly main phase conceptual base generation identify main problems objectives project scope systematic literature review conduct phase highlight relationships similarity measure among resources link data main issue feature type base link data solution development design develop experimental prototype test algorithms study thesis result main result obtain first systematic literature view base link data framework execute alyze recommendation algorithms base link data iii dynamic algorithm resource recommendation base knowledge link data relationships comparative study algorithms base link data two implementations propose framework one graph base algorithms machine learn algorithms application framework various scenarios demonstrate feasibility within context real applications conclusions propose framework demonstrate useful develop ing evaluate different configurations algorithms create novel base link data suitable users requirements applications domains contexts layer architecture propose framework also useful towards ',
 'advent call big data paradigm motivate flurry research aim enhance machine learn model follow diverse approach context work focus automatic construction feature supervise learn problems differ conventional selection feature new characteristics enhance predictive power infer original dataset particular manuscript propose new iterative feature construction approach base self learn meta heuristic algorithm harmony search solution encode strategy correspondingly cartesian genetic program suit represent combinations feature mean constant length solution vectors propose feature construction algorithm coin adaptive cartesian harmony search achs incorporate modifications allow exploit estimate predictive importance intermediate solutions ultimately attain better convergence rate iterative learn procedure performance propose achs scheme assess compare render state art toy example three practical use case literature excellent performance figure obtain problems shed light widespread applicability propose scheme supervise learn legacy datasets compose already refine characteristics work fund part basque government elkartek program bid project grant ref',
 'robot motion plan one central problems robotics receive considerable amount attention roboticists also control artificial intelligence communities despite different type applications physical properties robotic systems many high level task autonomous systems decompose subtasks require point point navigation avoid infeasible regions due obstacles workspace dissertation aim develop new class sample base motion plan algorithms fast efficient asymptotically optimal employ ideas machine learn dynamic program first interpret robot motion plan problem form machine learn problem since underlie search space know priori utilize random geometric graph compute consistent discretizations underlie continuous search space integrate exist algorithms algorithms framework sample base algorithms better exploitation exploration respectively introduce novel sample base algorithm call rrt improve upon well know rrt algorithm leverage value policy iteration methods new information collect propose algorithms yield provable guarantee correctness completeness asymptotic optimality also develop adaptive sample strategy consider exploration classification regression problem use online machine learn algorithms learn relevant region query region contain optimal solution without significant computational overhead extend application sample base algorithms class stochastic optimal control problems problems differential constraints specifically introduce path integral rrt algorithm solve optimal control stochastic systems rrt algorithm use close loop prediction trajectory generation differential systems one key benefit rrt many systems give low level track controller easier handle differential constraints complex steer procedures need unlike exist kinodynamic sample base algorithms implementation result sample base planners route plan full scale autonomous helicopter autonomous aerial cargo utility system program aacus program providework deal aspects support vector machine learn large scale data mine task base decomposition algorithm support vector machine train run serial well share memory parallel mode introduce transformation train data allow usage expensive generalize kernel without additional cost present experiment gaussian kernel usage kernel function possible order speed decomposition algorithm analyze critical problem work set selection large train data set addition analyze influence work set size onto scalability parallel decomposition scheme test conclusions lead several modifications algorithm improvement overall support vector machine learn performance method allow use extensive parameter search methods optimize classification accuracyconditional exponential model one widely use conditional model machine learn field improve iterative scale iis one major algorithms find optimal parameters conditional exponential model paper propose faster iterative algorithm name fis able find optimal parameters faster iis algorithm theoretical analysis show propose algorithm yield tighter bind traditional iis algorithm empirical study text classification three different datasets show new iterative scale algorithm converge substantially faster iis algorithm conjugate gradient algorithm furthermore examine quality optimal parameters find learn algorithm case incomplete train experiment show limit amount computation allow convergence achieve new algorithm fis able obtain lower test errors iis method methodpaper highlight new opportunities design large scale machine learn ing systems consequence blur traditional boundaries allow algorithm designers application level practitioners stay part oblivious detail underlie hardware level implementations hardware software design methodology advocate hinge deploy ment compute intensive machine learn kernels onto compute platforms trade determinism computation improvement speed energy efficiency achieve revisit digital stochastic circuit approximate matrix computations ubiquitous machine learn algorithms theoret ical empirical evaluation undertake assess impact hardware induce computational noise algorithm performance proof concept stochastic hardware simulator employ train deep neural network image recognition problemsdescribe software system enable tight integration vision control modules complex high dof humanoid robots demonstrate icub humanoid robot perform visual object detection reach grasp action key capability system reactive avoidance obstacle object detect video stream carry reach grasp task subsystems architecture independently improve update example show use machine learn techniques improve visual perception collect image robot interaction environment describe task software design constraints lead layer modular system architectureelectrical medical record restrict difficult centralize machine learn model train due privacy regulatory issue one solution train model distribute manner involve many party process however sometimes certain party trustable project aim propose alternative method traditional federate learn central analyzer order conduct train situation without trustable central analyzer propose algorithm call federate machine learn anonymous random hybridization abbreviate fearh use mainly hybridization algorithm degenerate integration connections medical record data model parameters add randomization parameter set share party base experiment new algorithm similar aucroc aucpr result compare machine learn centralize manner original federate machine learncurrently optimization reconfigurable design parameters typically do manually often involve substantial amount effort main focus thesis reduce effort designer focus implementation design correctness leave tool carry optimization address thesis make three main contributions udfirst present initial investigation reconfigurable design optimization machine learn optimizer mlo algorithm algorithm base surrogate model technology particle swarm optimization use surrogate model long hardware generation time mitigate automatic optimization possible first time best knowledge show model predict hardware generation fail well design perform udsecond introduce new algorithm call automatic reconfigurable design efficient global optimization ardego base efficient global optimization ego algorithm compare mlo support parallelism use simpler optimization loop ardego algorithm use multiple optimization compute nod optimization speed greatly improve relative mlo hardware generation time random nature two similar configurations take vastly different amount time generate make parallelization complicate novelty efficient use optimization compute nod achieve extension asynchronous parallel ego algorithm constrain problems udthird show result design synthesis benchmarking reuse design port different platform code revise achieve new auto transfer algorithm methodology make best use available synthesis benchmarking result novel contribution design automation reconfigurable systems open accesconditional exponential model one widely use conditional model machine learn field improve iterative scale iis one major algorithms find optimal parameters conditional exponential model paper propose faster iterative algorithm name fis able find optimal parameters faster iis algorithm theoretical analysis show propose algorithm yield tighter bind traditional iis algorithm empirical study text classification three different datasets show new iterative scale algorithm converge substantially faster iis algorithm conjugate gradient algorithm furthermore examine quality optimal parameters find learn algorithm case incomplete train experiment show limit amount computation allow convergence achieve new algorithm fis able obtain lower test errors iis method methodmachine learn model currently deploy variety real world applications model predictions use make decisions healthcare bank loan numerous critical task deployment artificial intelligence technologies become ubiquitous unsurprising adversaries begin develop methods manipulate machine learn model advantage visual analytics community develop methods open black box machine learn model little work focus help user understand model vulnerabilities context adversarial attack paper present visual analytics framework explain explore model vulnerabilities adversarial attack framework employ multi faceted visualization scheme design support analysis data poison attack perspective model data instance feature local structure demonstrate framework two case study binary classifiers illustrate model vulnerabilities respect vary attack strategiesgeometric perspective nonlinear binary classification algorithms include stateof art versions support vector machine svm radial basis function network rbfn classifiers base idea reconstruct indicator function pro pose instead use reconstruction sign distance function sdf basis binary classification discuss properties sign distance function exploit inclassification algorithms develop simple versions classifiers test several linear nonlinear problems linear test accuracy new algorithm exceed ofstandard svm methods average fewer misclassifications performance new methods also match exceed standard methods several nonlinear problemsincluding classification benchmark diagnostic micro array data set machine learn microarray data introduction binary classification basic problem machine learn applications many field binary classification many potential direct applications also basis many multi category classification methods particular interest applications biology medicine availability micro array proteomic data set contain thousands even tens thousands measurements particularly make important develop good classification algorithms since reliable use data could presumably revolutionize diagnostic medicine several binary classification algorithms develop study intensely past years notable among support vector machine svm methods use radial basis function function kernels svm methods show perform reasonably well classify micro array data demonstrate extraction useful information large data set feasiblepaper propose two new neuro fuzzy scheme one classification one cluster problems classification scheme base simpson fuzzy min max method relax assumptions make enable scheme handle mutually nonexclusive class neuro fuzzy cluster scheme multiresolution algorithm model mechanics human pattern recognition also present data exhaustive comparison techniques neural statistical machine learn traditional approach pattern recognition applications data set use comparisons include machine learn repository university california irvine find propose scheme compare quite well exist techniques addition offer advantage one pass learn online adaptationrecurrent neural network rnns widely use computational neuroscience machine learn ing applications rnn neuron com put output nonlinear function tegrated input importance rnns especially model brain process undis put also widely acknowledge com putations standard rnn model may simplification real neuronal network com pute suggest rnn approach may make neurobiologically plausi ble computationally powerful sion bayesian inference techniques non linear dynamical systems scheme use rnn generative model dynamic input cause environment speech kine matics give generative rnn model rive bayesian update equations decode output critically update define recogniz ing rnn rrnn neurons compute exchange prediction prediction error message rrnn several desirable feature con ventional rnn example fast cod dynamic stimuli robustness ini tial condition noise furthermore imple ments predictive cod scheme dynamic put suggest bayesian inversion recurrent neural network may useful model brain function machine learn tool illustrate use rrnn plication online decode recognition human kinematicspaper present detail evaluation set well know machine learn classifiers front dynamic non deterministic software anomalies system state prediction base monitor system metrics allow software proactive rejuvenation trigger automat ically random forest approach achieve validation errors less comparison well know algorithms avaluation order reduce automatically number monitor parameters need predict software anomalies analyze lasso regularization technique jointly machine learn ing classifiers evaluate prediction accuracy could guarantee within acceptable threshold allow reduce drastically around best case number monitor parameters framework base lasso regularization techniques validate use commerce environment apache tomcat server mysql database servercrucial part safety critical systems development identify system behaviours lead accidents complex systems hard need consider interaction multiple simultaneous failures normal system behaviours multi agent simulation provide way explore behaviour emerge model system normal failure condition however easy generate vast amount data simulations hard comprehend interpret machine learn apply help understand pattern implicit data paper first describe concept hazard analysis perform use multi agent simulation describe machine learn techniques use extract rule simulation output approach illustrate use military system systems case studyrecent advance sensor compute technology possible use real time machine learn techniques monitor state manufacture machine however make accurate predictions raw sensor data still difficult challenge work data process pipeline develop predict condition mill machine tool use raw sensor data acceleration audio time series sensor data aggregate block correspond individual cut operations computer numerical control cnc mill machine block data preprocessed use well know computationally efficient signal process techniques novel kernel function propose approximate covariance preprocessed block time series data several gaussian process regression model train predict tool condition different covariance kernel function model novel covariance function outperform model use common covariance function train model express use predictive model markup language pmml possible demonstrate predictive model component pipeline represent standardize form tool condition model show accurate especially predict condition lightly wear tool',
 'large datasets big data become ubiquitous cause potential value derive insights data across wide range business scientific applications increasingly recognize data growth accom panied rapid adoption large elastic multi tenant compute cluster compute cloud lead virtu ous cycle scalability cloud compute make possi ble analyze ever larger datasets proliferation big data lead adoption cloud compute particular machine learn one foundational dis ciplines data analysis summarization inference big data become routine organizations erate large cloud usually base systems hadoop support mapreduce program paradigm widely recognize mapreduce highly scal able suffer critical weakness machine learn ing support iteration consequently one program around limitation lead fragile inef ficient code reliance programmer inher ently flaw multi tenant cloud environment since programmer visibility state system program execute prior work seek address problem either develop spe cialized systems aim stylize applications aug menting mapreduce hoc support save state across iterations drive external loop paper advocate support loop first class construct propose extension mapreduce program paradigm call iterative mapreduce develop optimizer class iterative mapreduce program cover machine learn techniques provide theoretical justifications key optimization step empirically demonstrate system optimize program significant machine learn task competitive state art specialize solutionspresent new frank wolfe type algorithm applicable tominimization problems nonsmooth convex objective provide convergencebounds show scheme yield call coreset result variousmachine learn problems include median balance development sparse pca graph cut ell norm regularize support vector machine svm among others mean algorithm provide approximate solutions tothese problems time complexity bound dependent size ofthe input problem framework motivate grow body work onsublinear algorithms various data analysis problems entirelydeterministic make use smooth proximal operators apart fromthese theoretical result show experimentally algorithm verypractical case also offer significant computational advantage onlarge problem instance provide open source implementation beadapted problems fit overall structurecurrent age fourth industrial revolution industry digital world wealth data internet things iot data cybersecurity data mobile data business data social media data health data etc intelligently analyze data develop correspond smart automate applications knowledge artificial intelligence particularly machine learn key various type machine learn algorithms supervise unsupervised semi supervise reinforcement learn exist area besides deep learn part broader family machine learn methods intelligently analyze data large scale paper present comprehensive view machine learn algorithms apply enhance intelligence capabilities application thus study key contribution explain principles different machine learn techniques applicability various real world application domains cybersecurity systems smart cities healthcare commerce agriculture many also highlight challenge potential research directions base study overall paper aim serve reference point academia industry professionals well decision makers various real world situations application areas particularly technical point viewindustry aim make collaborative robotics accessible effective inside factories human robot interaction enhance mean advance perception systems allow flexible reliable production one contenders challenge intent improve cooperation industry within competition develop novel visual servoing system base machine learn technique automation wind copper wire production electric motor image base visual servoing systems often limit speed image process module run frequency order magnitude lower respect robot control speed article solution problem propose visual servoing function synthesize use gaussian mixture model gmm machine learn system guarantee extremely fast response issue relate data size reduction collection data set need properly train learner discuss performance propose method compare standard visual servoing algorithm use train gmm system develop test path follow application aluminium bar simulate real stator teeth generic electric motor experimental result demonstrate propose method able reproduce visual servoing function minimal error guarantee extremely high work frequencyfast coreset minimum enclose ball kernel algorithm propose first transfer kernel methods center constrain minimum enclose ball problem subsequently train kernel methods use propose meb algorithm primal variables kernel methods recover via kkt condition detail theoretical analysis rigid proof new algorithm give experiment investigate via use several typical classification datasets uci machine learn benchmark datasets moreover performances compare standard support vector machine seriously consider conclude propose algorithm own comparable even superior performances yet rather fast converge speed experiment study paper finally comment exist problems future development directions discuss school hotel tourism management academic research referee referee conference papework deal aspects support vector machine learn large scale data mine task base decomposition algorithm support vector machine train run serial well share memory parallel mode introduce transformation train data allow usage expensive generalize kernel without additional cost present experiment gaussian kernel usage kernel function possible order speed decomposition algorithm analyze critical problem work set selection large train data set addition analyze influence work set size onto scalability parallel decomposition scheme test conclusions lead several modifications algorithm improvement overall support vector machine learn performance method allow use extensive parameter search methods optimize classification accuracybegin machine learn methodology origin artificial intelligence rapidly spread different research communities successful outcomes chapter aim introduce system analysers designers comparatively new statistical supervise machine learn algorithm call support vector machine svm explain two useful areas svm classification regression basic mathematical formulation simple demonstration make easy understand svm prospect challenge future research emerge area also describe future research svm provide improve quality access users therefore develop automate svm system state art technologies paramount importance hence chapter link important step system analysis design perspective evolve research arenasensors existence key component cyber physical systems make susceptible failures due complex environments low quality production age defective sensors either stop communicate convey incorrect information unsteady situations threaten safety economy reliability system objective study construct lightweight machine learn base fault detection diagnostic system within limit energy resources memory computation wireless sensor network wsn paper context aware fault diagnostic cafd scheme propose base ensemble learn algorithm call extra tree evaluate performance propose scheme realistic wsn scenario compose humidity temperature sensor observations replicate extreme low intensity fault six commonly occur type sensor fault consider drift hard bias spike erratic precision degradation stick data loss propose cafd scheme reveal ability accurately detect diagnose low intensity sensor fault timely manner moreover efficiency extra tree algorithm term diagnostic accuracy score roc auc train time demonstrate comparison cut edge machine learn algorithms support vector machine neural networkpaper cluster base machine learn algorithm call cluster algorithm system cas introduce cas algorithm test evaluate performance find fruitful result present heuristics facilitate machine learn author boost research work infobase ministry civil service use analyze cas algorithm cas algorithm compare machine learn algorithms like unimem cobweb classit find strong point propose algorithm combine advantage two different approach machine learn first approach learn examples cas support single multiple inheritance exceptions cas also avoid probability assumptions well understand concept formation second approach learn observation cas apply set operators prove effective conceptual cluster show cas build search cluster hierarchy incorporate characterize objectdissertation present four application drive robotic manipulation task solve use combination feature base machine learn dimensionality reduction optimization techniques first study previously publish image process algorithm whose goal learn classify pixels image consider good bad grasp point exploit ideas behind dimensionality reduction general principal component analysis particular formulate feature selection search space reduction hypotheses provide approach reduce algorithm computation time retain classification accuracy second incorporate image process technique new method compute valid end effector orientations grasp task combination generate unimanual rigid object grasp planner specifically fast accurate three layer hierarchical supervise machine learn framework develop robot kinesthetically teach set valid end effector orientations human loop third solve challenge bimanual regrasping pick place operation require object transfer one manipulator another cast optimization problem objective minimize execution time optimization problem supplement image process unimanual grasp algorithm jointly identify two good grasp point object proper orientations end effector fourth target deformable object solve problem use cooperative manipulators perform towel fold task solve problem new learn algorithm combine imitation reinforcement learn way human demonstrations use reduce search space reinforcement learn algorithm result quick convergence fast learn capabilities collectively task solve dissertation establish application orient feature base machine learn techniques robotics although task different range unimanual bimanual manipulation handle rigid deformable object mathematical frameworks design principles behind implementations similar addition common use feature machine learn dimensionality reduction task commonly design general efficient modular anthropomorphic manipulator end effector sensor independent properties affect choices make algorithms development also allevipaper highlight new opportunities design large scale machinelearning systems consequence blur traditional boundaries haveallowed algorithm designers application level practitioners stay forthe part oblivious detail underlie hardware levelimplementations hardware software design methodology advocate herehinges deployment compute intensive machine learn kernels ontocompute platforms trade determinism computation improvementin speed energy efficiency achieve revisit digitalstochastic circuit approximate matrix computations ubiquitousin machine learn algorithms theoretical empirical evaluation isundertaken assess impact hardware induce computational noise onalgorithm performance proof concept stochastic hardware simulatoris employ train deep neural network image recognition problemspaper propose concept machine learn integrate grid scheme least square support vector machine lssvm call lssvm mix kernel order solve data classification problems purpose lssvm execute feature selections mix kernel applications parameter optimization learn paradigm propose learn paradigm include three step first orthogonal design utilize initialize number input feature candidate parameters store feature randomly select accord first grid acquire first step feature candidate parameters pass lssvm finally artificial bee colony algorithm recently popular heuristic algorithm use optimize parameters lssvm learn illustration evaluation purpose ten remarkable data set university california irvine database use test target experimental result reveal propose lssvm produce classification model easily interpret use small number feature term accuracy hit ratio lssvm significantly outperform methods list paper find imply lssvm promise approach classification explorationemergence competency base train current evaluation scheme surgical skills evolve include newer methods assessment train artificial intelligence machine learn algorithms utilize extensive data set analyze operator performance study aim address question artificial intelligence uncover novel metrics surgical performance support vector machine algorithms train differentiate senior junior participants execute virtual reality hemilaminectomy algorithms achieve good classification performance participants canadian universities divide group accord train level senior junior ask perform virtual reality hemilaminectomy position angle force application simulate burr suction instrument along tissue volumes remove record intervals raw data manipulate create metrics train machine learn algorithms five algorithms include support vector machine train predict whether task perform senior junior participant accuracy algorithm assess leave one cross validation forty one individuals enrol senior junior participants twelve metrics relate safety procedure efficiency motion tool coordination select follow cross validation support vector machine achieve accuracy algorithms achieve accuracy respectively artificial intelligence define novel metrics surgical performance outline train level virtual reality spinal simulation procedure significance result lie potential artificial intelligence complement current educational paradigms better prepare residents surgical proceduresmachine learn one important subfields computer science use solve variety interest artificial intelligence problems different languages framework tool define data need solve machine learn base problems however great number diverse alternatives make difficult intercommunication portability usability definitions design algorithms developer may create paper take first step towards language development environment independent underlie technologies allow developers design solutions solve machine learn base problems simple fast way automatically generate code technologies consider transparent bridge among current technologies rely model drive engineer approach focus creation model definition artifacts underlie technologiesconditional exponential model one widely use conditional model machine learn field improve iterative scale iis one major algorithms find optimal parameters conditional exponential model paper propose faster iterative algorithm name fis able find optimal parameters faster iis algorithm theoretical analysis show propose algorithm yield tighter bind traditional iis algorithm empirical study text classification three different datasets show new iterative scale algorithm converge substantially faster iis algorithm conjugate gradient algorithm furthermore examine quality optimal parameters find learn algorithm case incomplete train experiment show limit amount computation allow convergence achieve new algorithm fis able obtain lower test errors iis method methodthesis consist four chapters chapter focus theoretical result high order laplacian base regularization function estimation study iterate laplacian regularization context supervise learn order achieve nice theoretical properties like thin plate splines good performance complex region like soap film smoother chapter propose innovative static path plan algorithm call within environment full obstacles theoretically show reduce number vertex simulation study approach outperform arm standard heuristic stronger ones true distance heuristics tdh yield faster query time adequate usage memory reasonable preprocessing time chapter propose lpa algorithm extend algorithm context dynamic path plan achieve better performance compare benchmark lifelong plan lpa term robustness worst case computational complexity employ beamlet graphical structure lpa encode information environment hierarchical multiscale fashion therefore produce robust dynamic path plan algorithm chapter focus approach prediction spot electricity spike via combination boost wavelet analysis extensive numerical experiment show approach improve prediction accuracy compare result support vector machine thank fact gradient boost tree method inherit good properties decision tree robustness irrelevant covariates fast computational capability good interpretation phdcommittee chair huo xiaoming committee member deng shi jie committee member shapiro alex committee member tsiotras panagiotras committee member yuan minevolve area cybersecurity present dynamic battlefield cyber criminals security experts intrusions become major concern cyberspace different methods employ tackle threats need ever update traditional methods rudimentary approach manually update blacklist whitelists another method involve manually create rule usually one common methods date lot similar research involve incorporate machine learn artificial intelligence host network base intrusion systems recently originally present problems low accuracy growth area machine learn last decade lead vast improvements machine learn algorithms requirements research apply nearest neighbour fold cross validation random forest machine learn algorithms network base intrusion detection system order improve accuracy intrusion detection system project focus specific feature selection improve increase detection accuracy use fold cross validation algorithm random forest algorithm approximately sample nsl kdd datasetgenetic algorithms random heuristic search rhs algorithms adaptive systems wide range applications search optimisation pattern recognition machine learn well signal process despite widespread use general theory still lack promise approach offer dynamical system model describe stochastic trajectory population dynamics genetic algorithm help underlie deterministic heuristic function fix point however even simple genetic algorithm sga fitnessproportional selection crossover mutation determination population trajectory fix point heuristic function unfeasible practical problem size order simplify mathematical analysis selection introduce paper base strong selection scheme possible derive dynamical system model respective fix point close form addition theoretical analysis experimental result present index term genetic algorithm selection random heuristic search dynamical system modelsupport vector machine popular method machine learn incremental support vector machine algorithm ideal selection face large learn data set paper new incremental support vector machine learn algorithm propose improve efficiency large scale data process model incremental learn algorithm similar standard support vector machine goal concept update incremental learn train procedure include new train data time complexity independent whole train set compare incremental version train speed approach improve change hyperplane reducemachine learn base indoor localization use suffer collection construction maintenance label train databases practical implementation semi supervise learn methods develop efficient indoor localization methods reduce use label train data boost efficiency accuracy indoor localization paper propose new time series semi supervise learn algorithm key aspect develop method distinguish conventional semi supervise algorithms use unlabeled data learn algorithm find spatio temporal relationships unlabeled data pseudolabels generate compensate lack label train data next step another balance optimization learn algorithm learn position model propose method evaluate estimate location smartphone user use receive signal strength indicator rssi measurement experimental result show develop learn algorithm outperform exist semi supervise algorithms accord variation number train data access point also propose method discuss term give better performance analysis impact learn parameters moreover extend localization scheme conjunction particle filter execute include additional information floor planconcept boost emerge field machine learn basicidea boost accuracy weak classify tool combine variousinstances accurate prediction general concept lateradapted field statistical model review article attempt tohighlight evolution boost algorithms machine learn tostatistical model describe adaboost algorithm classification aswell two prominent statistical boost approach gradientboosting likelihood base boost although appraoches typicallytreated separately literature share methodological rootsand follow fundamental concepts compare initial machinelearning algorithms must see black box prediction scheme statistical boost result statistical model offer astraight forward interpretation highlight methodological background andpresent common software implementations work examples andcorresponding code find appendixcore data structure fca formal concept analysis concept lattice widely use field machine learn data mine useful study algorithms build concept lattice practical applications several algorithms build concept lattice develop paper present efficient algorithm name cmcg concept matrix base concepts generation build concept lattice correspond hasse graph base concept matrix novel notion algorithm cmcg find lower neighbor concept use rank attribute concept matrix generate correspond hasse graph validity algorithm prove theory experiment pseudo cod cmcg algorithm give performance cmcg superior one lattice algorithm prove endobjective study apply machine learn algorithms real time personalize wait time prediction emergency departments also aim introduce concept systems think enhance performance prediction model four popular algorithms apply stepwise multiple linear regression artificial neural network iii support vector machine gradient boost machine linear regression model serve baseline model comparison conduct computational experiment base dataset collect emergency department hong kong model diagnostics perform result cross validate four machine learn algorithms use systems knowledge outperform baseline model stepwise multiple linear regression reduce mean square error almost three algorithms similar performances reduce mean square error approximately reductions mean square error due utilization systems knowledge observe multi dimensional stochasticity arise environment impose great challenge wait time prediction introduction concept systems think lead significant enhancements model suggest interdisciplinary efforts could potentially improve prediction performance machine learn algorithms utilization systems knowledge could significantly improve performance wait time prediction wait time prediction less urgent patients challengesequential monte carlo smc method simulation base approach compute posterior distributions smc methods often work well applications consider intractable methods due high dimensionality computationally demand smc implement efficiently fpgas design productivity remain challenge paper introduce design flow generate efficient implementation reconfigurable smc design templating smc structure design flow enable efficient map smc applications multiple fpgas propose design flow consist parametrisable smc computation engine open source software tem plate enable efficient map variety smc design reconfigurable hardware design parameters critical performance solution quality tune use machine learn algorithm base surrogate model experimental result three case study show design performance substantially improve rameter optimisation propose design flow demonstrate capability produce reconfigurable implementations range smc applications significant improvement speed energy efficiency optimise cpu gpu implementations keywords fpga sequential monte carlo machine learn ingpaper review quality sim ilarity measure applications machine learn ing present measure analyze perspective granular compute granular compute allow analyze information differ ent level different approach analysis show measure base two basic aspects universe object granular ity information principle similar problems similar solutions use measure method formulate build relations similar ity relations result use improve machine learn techniqueseffective defect detection still hot issue come software quality assurance static source code analysis play thereby important role since offer possibility automate defect detection early stag development detect defect see classification problem machine learn recently investigate use purpose study present new model automate defect detection mean machine learn ers base static java code feature model comprise extraction necessary feature well application suitable classifiers realize prototype feature extraction study prototype output order identify suitable classifiers finally overall approach evaluate use open source project suitability study evaluation show several classifiers suitable model rotation forest multilayer perceptron jrip classifier make approach effective detect defect accuracy higher although approach comprise prototype show potential become effective alternative nowa days defect detection methodsrecent years grow number searchers investigate performance machine learn ing base traffic classification use statistical properties classification techniques require packet payload inspection techniques assist internet service providers work within legal technical limitations direct payload inspection potential new applications include automate market research automate traffic prioritisation lawful interception many new applications couple flow classifi cation subsequent flow treatment block shape highly desirable develop diffuse extensions exist packet filter provide base traffic classification base statistical properties couple flow classification flow treatment report describe selection exist packet filter extend design overall architecture key components well machine learn techniqueintroductory tutorial paper special session machine learn speak dialogue systems last decade research field speak dialogue sys tems sds experience increase growth yet sign optimization sds consist combin ing speech language process systems automatic speech recognition asr parsers natural language gener ation nlg text speech tts synthesis systems also require development dialogue strategies take least account performances subsystems others nature task form fill tutor robot control database search browse user behaviour cooperativeness expertise due great variability factor reuse previous hand craft design also make difficult reason statistical machine learn ing methods apply automatic sds optimization lead research area last years paper provide short review field recent advanceapproach common machine learn ing literature know relevance feedback learn apply provide method manage agile sensors context machine learn application image retrieval relevance feedback proceed follow user goal image mind retrieve database image learn system system compute image set image display query oftentimes decision image display do use divergence metrics kullback leibler divergence user indicate relevance image goal image system update estimate typically probability mass function database image procedure repeat desire image find method manage agile sensors proceed analogous manner goal system learn number state group move target occupy surveillance region system compute sense action take query base divergence measure call nyi divergence measurement make provide relevance feedback system update probability density number state target procedure repeat time sensor available use show use simulate measurements real record target trajectories method sensor management yield ten fold gain sensor efficiency compare periodic scanchallenge test machine learn applica tions intend learn properties data set correct answer already know absence test oracle one approach test plications use metamorphic test proper tie application exploit define transforma tion function input new output unchanged easily predict base original output output expect defect must exist application seek enumerate classify metamorphic properties machine learn ing algorithms demonstrate apply reveal defect applications interest addition result test present set properties use define metamorphic relationships metamorphic test use general proach test machine learn applicationsweb languages machine learn adaptive communicator heuristics problems program artiftcial intelligence flexible interface mechanism computers user application program thip report consider adaptive communi ator interpose application program set input cutput devices communicator couled devices program transform raw signal low level logical object communicator envision series context analysis systems separate context set operate rule communicate transmit logical object report conclude label direct graph web appropriate data base organization man machine interaction web grammars form pattern replacement rule use manipulate data base pattern replacement rule view heuristics suitable machine learn ing arpa arpenergy prediction machine tool deliver many advantage manufacture enterprise range energy efficient process plan machine tool monitor physics base energy prediction model propose past understand energy usage pattern machine tool however uncertainties machine operate environment make difficult predict energy consumption target machine reliably take advantage opportunity collect extensive contextual energy consumption data discuss data drive approach develop energy prediction model machine tool paper first present methodology efficiently effectively collect process data extract machine tool sensors present data drive model use predict energy consumption machine tool machine generic part specifically use gaussian process regression non parametric machine learn technique develop prediction model energy prediction model generalize multiple process parameters operations finally apply generalize model method assess uncertainty intervals predict energy consume machine part use mori seiki nvd machine tool furthermore model use process plan optimize energy efficiency machine process',
 'conditional exponential model one widely use conditional model machine learn field improve iterative scale iis one major algorithms find optimal parameters conditional exponential model paper propose faster iterative algorithm name fis able find optimal parameters faster iis algorithm theoretical analysis show propose algorithm yield tighter bind traditional iis algorithm empirical study text classification three different datasets show new iterative scale algorithm converge substantially faster iis algorithm conjugate gradient algorithm furthermore examine quality optimal parameters find learn algorithm case incomplete train experiment show limit amount computation allow convergence achieve new algorithm fis able obtain lower test errors iis method methodmax product algorithm local message pass scheme attempt tocompute probable assignment map give probability distribution successfully employ method approximate inference forapplications arise cod theory computer vision machine learn however max product algorithm guarantee converge mapassignment guarantee recover map assignment alternative convergent message pass scheme propose overcomethese difficulties work provide systematic study suchmessage pass algorithms extend know result exhibit newsufficient condition convergence local global optima providinga combinatorial characterization optima base graph cover anddescribing new convergent correct message pass algorithm whosederivation unify many know convergent message pass algorithms convergent correct message pass algorithms represent stepforward analysis max product style message pass algorithms theconditions need guarantee convergence global optimum toorestrictive theory practice limitation convergent andcorrect message pass scheme characterize graph cover andillustrated example comment complete rework expansion previous versionone class classification problem often address solve constrain quadratic optimization problem spirit support vector machine paper derive novel one class classification approach investigate original sparsification criterion criterion know coherence criterion base fundamental quantity describe behavior dictionaries sparse approximation problems propose framework allow derive new theoretical result associate coherence criterion one class classification algorithm solve least square optimization problem also provide adaptive update scheme experiment conduct real datasets time series illustrate relevance approach exist methods accuracy computational efficiency index term support vector machine machine learn kernel methods one class classificationunambiguous algorithm add study applicability domain appropriate measure goodness fit robustness represent key characteristics ideally fulfil qsar model consider regulatory purpose paper propose new algorithm rinh base rivality index construction qsar classification model index capable predict activity data set molecules mean measurement rivality nearest neighbor belong different class contribute robust measurement reliability predictions order demonstrate goodness propose algorithm select four independent orthogonally different benchmark data set balance unbalance high low modelable compare result obtain use different machine learn algorithms result validate use data set different balance size corroborate propose algorithm able generate highly accurate classification model contribute valuable measurements reliability predictions applicability domain build modelactive machine learn algorithms use large number unlabeled examples available get label costly require consult human expert many conventional active learn algorithms focus refine decision boundary expense explore new regions current hypothesis misclassifies propose new active learn algorithm balance exploration refine decision boundary dynamically adjust probability explore step experimental result demonstrate improve performance data set require extensive exploration remain competitive data set algorithm also show significant tolerance noisemachine learn invoke imagination many scientific mind due potential solve complex difficult real world problems paper give methods construct machine learn tool use support vector machine svms first give simple example illustrate basic concept demonstrate practical problem practical problem concern electronic monitor fishways automatic count different fish species purpose environmental management australian rivers result illustrate power svm approach sample problem computational attractiveness practical implementations december open accesmachine learn concept incorporate number software devices computer science information industry software devices capable decision make like human brain capability decision make govern artificial intelligence techniques techniques follow many algorithms develop decision make machine learn decision make depend upon profound train contemporary data particular domain data play major important part one element machine learn algorithm main focus paper develop machine learn algorithm help train available medical domain data prepare data model negotiate query achieve analysis different machine learn methodologies like support vector machine svm decision tree recursive partition algorithm model build process new data mine machine learn algorithm propose along performance analysis medical domain dataset analysis indicate data size increase continuous increase algorithm accuracy concurrently time consumption also increase general term classification algorithinternational audiencein many machine learn settings label examples difficult collect unlabeled data abundant also binary classification problems positive examples examples target class available additional data use improve accuracy supervise learn algorithms investigate paper design learn algorithms positive unlabeled data many machine learn data mine algorithms decision tree induction algorithms naive bay algorithms use examples order evaluate statistical query like algorithms kearns design statistical query learn model order describe algorithms design algorithm scheme transform like algorithm algorithm base positive statistical query estimate probabilities set positive instance instance statistical query estimate probabilities instance space prove class learnable statistical query learn model learnable positive statistical query instance statistical query lower bind weight target concept estimate polynomial time design decision tree induction algorithm posc base use positive unlabeled examples give experimental result algorithm case imbalanced class sense one two class say positive class heavily underrepresented compare class remain open problem challenge encounter many real world applicationsdeliver fault free code clear goal devel oper however best method achieve aim still open question despite several approach propose literature exist overall best way one possible solution propose recently combine static source code analysis discipline machine learn ing approach direction define within work implement prototype validate subse quently show possible translation piece source code machine learn algorithm input suitability task fault detection context present work two prototypes veloped show feasibility present idea output generate open source project collect use train rank various machine learn ing classifiers term accuracy false positive false negative rat best among subsequently validate open source project first study least classifiers include multilayerper ceptron ibk adaboost bftree could convince except latter fail completely could validate second study despite prototype show suitability machine learn algorithms static source code analysisterm machine learn refer collection tool use identify pattern data oppose traditional methods pattern identification machine learn tool rely artificial intelligence map patter large amount data self improve new data become available quicker accomplish task review describe various techniques machine learn use past prediction detection management infectious diseases tool bring battle covid addition also discuss applications various stag pandemic advantage disadvantage possible pit falldue latest advancements monitor technologies interest possibility early detection quality issue components grow considerably manufacture industry however implementation techniques limit outside research environment due demand scenarios pose production environments paper propose method assess health machine process machine tool apply range machine learn techniques sensor data aim work provide complete diagnosis condition provide rapid indication machine tool process change beyond acceptable limit make realistic solution production environments prior research author find good visibility simulate failure modes number machine operations machine tool fingerprint routines define sensor suite current research set utilise system streamline test procedure obtain large dataset test techniques upon various supervise unsupervised techniques implement use range feature extract raw sensor signal principal component analysis continuous wavelet transform latter classify use convolutional neural network cnn custom make network pre train network transfer learn detection classification accuracies simulate failure modes across classical cnn techniques test promise approach certain conditionaccord standard procedure build classier use machine learn fully automate process follow preparation train data domain expert contrast interactive machine learn engage users actually generat ing classier oers natural way integrate background knowledge model stage long interactive tool design support cient eective communication paper show appropriate techniques empower users create model compete classiers build state art learn algorithms demonstrate users even users domain experts often construct good classiers without help learn algorithm use simple two dimensional visual interface experiment real data demonstrate surprisingly success hinge domain attribute support good predictions users generate accurate classiers whereas domains many high order attribute interactions favor standard machine learn ing techniques also present articial example domain knowledge allow expert user create much accurate model automatic learn algorithms result indicate system potential produce highly accurate classiers hand domain expert strong interest domain therefore insights partition data moreover small expert dened model oer additional advantage generally intelligible generate automatic techniques',
 'paper present integrate hybrid optimization algorithm train radial basis function neural network rbf train neural network still challenge exercise machine learn domain traditional train algorithms general suffer trap local optima lead premature convergence make ineffective apply datasets diverse feature train algorithms base evolutionary computations become popular due robust nature overcome drawbacks traditional algorithms accordingly paper propose hybrid train procedure differential search algorithm functionally integrate particle swarm optimization pso surmount local trap search procedure new population initialization scheme propose use logistic chaotic sequence enhance population diversity aid search capability demonstrate effectiveness propose rbf hybrid train algorithm experimental analysis publicly available benchmark datasets perform subsequently experiment conduct practical application case wind speed prediction expound superiority propose rbf train algorithm term prediction accuracyability witness nonlocal correlations lie core foundational aspects quantum mechanics application process information commonly achieve via violation bell inequalities unfortunately however systematic derivation quickly become unfeasible scenario interest grow complexity cope propose machine learn approach detection quantification nonlocality consist ensemble multilayer perceptrons blend genetic algorithms achieve high performance number relevant bell scenarios show machine learn quantify nonlocality discover new kinds nonlocal correlations inaccessible current methods well also apply framework distinguish classical quantum even postquantum correlations result offer novel method proof principle relevance machine learn understand nonlocality',
 'machine learn data mine artificial intelligence big data increasingly apply psychological science although areas research benefit tremendously new set statistical tool often use biological genetic variables hype substantiate traditional areas research argue phenomenon result measurement errors prevent machine learn algorithms accurately model nonlinear relationships indeed exist shortcoming showcased across set simulate examples demonstrate model selection machine learn algorithm regression depend measurement quality regardless sample size conclude set recommendations discussion ways better integrate machine learn statistics traditionally practice psychological sciencealgorithm machine learn algorithm new hybrid intelligent algorithm use solve resources schedule problem new algorithm contain adaptive particle swarm optimization apso algorithm modify genetic algorithm mga machine learn algorithm mga use realize global search apso use get local search choose process depend definite information ant algorithm machine learn principle propose iteration part optimal solution deserve search optimal solution layer simulational result base well know benchmark suit literature show algorithm better optimization performancerobot program demonstration intuitive method program robot programmer show particular task perform use interface device allow measurement record human motion parameters relevant perform demonstrate task paper present analysis learn interaction requirements characteristic rpd system base requirements new system architecture propose support phase interactive program process example task experimental result give keywords program demonstration man machine interface machine learn introduction due cost involve development maintenance robot program automatic program techniques program human demonstration pbd currently attract lot interest moreover learn capabilities become continuously attractivepaper deal concepts expert system data mine belong artificial intelligence field main task expert system ratiocination machine learn algorithm find better optimal solution paper mainly focus diagnose disease effect emu bird mechanism particle swarm optimization pso algorithm artificial bee colony abc algorithm decisive rule database mine could apply expert system thus apply optimization techniques result best global optimize solution keywords optimization global optimal solution machine learn algorithm expert system decisive rulemachine learn branch artificial intelligence increasingly use health research include nurse maternal outcomes research machine learn algorithms complex involve statistics terminology common health research purpose methods paper describe three machine learn algorithms detail provide example use maternal outcomes research three algorithms classification regression tree least absolute shrinkage selection operator random forest may use understand risk group select variables model rank variables contribution outcome respectively machine learn plenty contribute health research also drawbacks discuss well provide example different algorithms function use complete cross sectional study examine association oxytocin total dose exposure primary cesarean section result algorithms compare do find use traditional methodsartificial intelligence broad set sophisticate computer base statistical tool become widely available cardiovascular medicine large data repositories need operational efficiency grow focus precision care set transform artificial intelligence applications range new pathophysiologic discoveries decision support individual patient care optimization system wide logistical process machine learn dominant form artificial intelligence wherein complex statistical algorithms learn deduce pattern datasets supervise machine learn use classify large data train algorithm accurately predict outcome whereas unsupervised machine learn algorithm uncover mathematical relationships within unclassified data artificial multilayered neural network deep learn one successful tool artificial intelligence demonstrate superior efficacy disease phenomapping early warn systems risk prediction automate process interpretation image increase operational efficiency artificial intelligence demonstrate ability learn assimilation large datasets unravel complex relationships discover prior unfound pathophysiological state develop predictive model artificial intelligence need widespread exploration adoption large scale implementation cardiovascular practicemotion sensor model crucial components current algorithms mobile robot localization map model typically provide hand tune human operator often derive intensive careful calibration experiment operator knowledge experience robot operate environment paper demonstrate parameters motion sensor model automatically estimate normal robot operations via machine learn methods thereby eliminate necessity manually tune model laborious calibration process result real world robotic experiment present show effectiveness estimation approachbachelor thesis deal application machine learn algorithms generation check circuit contain detail descriptions individual algorithms machine learn choose achieve purpose except familiarization theoretical properties also part devote specific utilization mention algorithms form classifiers classifiers work different settings influence accuracy learn subsequent classification differences individual classifiers settings illustrate experimental part thesis experiment conduct various circuit include control units robot develop department computer systems faculty information technology brno university technologyinternational audiencein real world robotic applications many factor low level vision motion control parameters high level behaviors determine quality robot performance thus many task robots require fine tune parameters implementation behaviors basic control action well strategic deci sional process recent years machine learn techniques use find optimal parameter set different behaviors however drawback learn techniques time consumption practical applications methods design physical robots must effective small amount data paper present method concurrent learn best strategy optimal parameters extend policy gradient reinforcement learn algorithm result experimental work simulate environment real robot show high convergence ratereal world robotic applications many factor low level vision motion control parameters high level behaviors determine quality robot performance thus many task robots quire fine tune parameters implementation behaviors basic control action well strategic deci sional process recent years machine learn techniques use find optimal parameter set different behaviors however drawback learn techniques time consumption practical applications methods design physical robots must effective small amount data paper present method concurrent learn best strategy optimal parameters extend policy gradient reinforcement learn algorithm result experimental work simulate environment real robot show high convergence rateresearchers discover many successful algorithms methodologies solve problems intersection machine learn education research umbrella category educational data mine enjoy series successes span research process post hoc data analysis generate model use model successful educational interventions however successes arise use pre exist psychological educational construct guess thus use semi supervise fully supervise machine learn algorithms algorithms novel discovery also know unsupervised cluster enjoy significantly fewer successes domain partially education data exhibit unique complex structure thesis mixture algorithm development simulation experimentation real world data design define test novel paradigm cluster education range domains paradigm target cluster revolve around inclusion high level target student learn pre test post test approach differ exist machine learn approach design completely initial concept final execution solve educational research problems take advantage structural complexities problematic algorithms thesis include range data set draw variety research domains include new data experiment psychological sense however thesis include analysis methodology result implications educational research perspective rely entirely education data research problemsnowadays big data environment conventional compute platform base von neumann architecture encounter bottleneck increase requirement computation capability efficiency brain inspire compute neuromorphic compute demonstrate great potential revolutionize technology world consider one promise solutions achieve tremendous compute power efficiency single chip neuromorphic compute systems represent great promise many scientific intelligent applications many design propose realize traditional cmos technology however progress slow recently rebirth neuromorphic compute inspire development novel nanotechnology thesis propose neuromorphic compute systems reram memristor crossbar array include work three major part memristor devices model relate circuit design resistive memory reram technology investigate physical mechanism statistical analysis intrinsic challenge weight sense scheme assign different weight cells different bite line propose area power overhead peripheral circuitry effectively reduce minimize amplitude sneak paths neuromorphic compute system design leverage memristor devices algorithm scale neural network machine learn algorithms base similarity memristive effect biological synaptic behavior first spike neural network snn rate cod model develop algorithm level map hardware design supervise learn addition speed accuracy improvement another neuromorphic system adopt analog input signal different voltage amplitude current sense scheme build moreover use single memristor crossbar neural net work layer explore application specific optimization reliability improvement develop neuromorphic systems thesis impact device failure memristor base neuromorphic compute systems cognitive applications evaluate retrain remapping design algorithm level hardware level develop rescue large accuracy lossquantum compute artificial intelligence combine together may revolutionize future technologies significant school think regard artificial intelligence base generative model propose general quantum algorithm machine learn base quantum generative model prove propose model capable represent probability distributions compare classical generative model exponential speedup learn inference least instance quantum computer efficiently simulate classically result open new direction quantum machine learn offer remarkable example quantum algorithm show exponential improvement classical algorithms important application fieldopus branch bind search algorithm enable efficient systematic search space order search operators apply significant opus achieve maximise effect prune action possible guarantee general case prune shall occur prune possible effect maximise experimental application opus data drive machine learn demonstrate hard search problems possible guarantee solution reasonable time solve real world data within acceptable time frame indeed opus demonstrate enable systematic search extremely large search space less time take common heuristic machine learn search algorithms use systematic search concept learn enable better experimental comparison alternative inductive bias previously possible precise inductive bias describeincreasingly clear machine learn algorithms need integrate iterative scientific discovery loop data query repeatedly mean inductive query computer provide guidance experiment perform chapter summarise several key challenge achieve integration machine learn data mine algorithms methods discovery quantitative structure activity relationships qsars introduce concept robot scientist step discovery process automate discuss representation molecular data knowledge discovery tool analyse discuss adaptation machine learn data mine algorithms guide qsar experimentmodern clinical practice require integration interpretation ever expand volumes clinical data therefore imperative develop efficient ways process understand large amount data neurologists work understand function biological neural network artificial neural network form machine learn algorithm likely increasingly encounter clinical practice use increase clinicians need understand basic principles common type algorithm aim provide coherent introduction jargon heavy subject equip neurologists tool understand critically appraise apply insights burgeon fieldfield data science continue grow anever increase demand tool make machine learn accessible tonon experts paper introduce concept tree base pipelineoptimization automate one tedious part machinelearning pipeline design implement open source tree base pipelineoptimization tool tpot python demonstrate effectiveness aseries simulate real world benchmark data set particular showthat tpot design machine learn pipelines provide significantimprovement basic machine learn analysis require little noinput prior knowledge user also address tendency tpotto design overly complex pipelines integrate pareto optimization whichproduces compact pipelines without sacrifice classification accuracy assuch work represent important step toward fully automate machinelearning pipeline design comment page figure preprint appear gecco edit yet make reviewer commentdata mine machine learn experimental sciences lot insight behaviour algorithms obtain implement study behave run datasets paper present new experimental methodology base concept experiment databases experiment database see special kind inductive database experimental methodology consist fill query database show novel methodology numerous advantage exist one paper present novel interest application inductive databases may significant impact experimental research machine learn data minedata mine machine learn experimental sciences lot insight behaviour algorithms obtain implement study behave run datasets however experiment often extensive systematic ideally would therefore experimental result must interpret caution paper present new experimental methodology base concept experiment databases experiment database see special kind inductive database experimental methodology consist fill query database show novel methodology numerous advantage exist one paper present novel interest application inductive databases may significant impact experimental research machine learn data mine',
 'daily task drill laser mark spot weldingapplication cartesian robot request reach hand tip desiredtarget location task become complex handle multiple pointsin shortest travel time space reason study wasconducted primary objective develop computational intelligent systemthat would contribute towards encourage productive quality way materialhandling process objective project design develop andoptimize performance cartesian robotic arm term position andspeed perform spot weld application genetic algorithm beintroduce able look optimum sequence solve path planningvia evolutionary solutions determine best combination paths order tominimize total motion weld time shortest travel distance newalgorithm test implement cartesian robot laser pointer willreplace spot weld torch demonstration purpose project thisproject involve develop machine learn system capable ofperforming independent learn capability give task design anddevelopment project involve two major section first section concernsabout hardware construction wire test second section involvessoftware design control movement robot spot weld thehardware design categorize two aspects electrical design andmechanical design electrical design involve wire control components suchas stepper motor controller input output devices well power supplyand safety devices finally develop algorithm test andimplemented cartesian robot systemrobots employ carry well define task customize static environments often high speed astonish level precision however robots usually depend totally action detail control scheme develop advance line plan phase due recent progress electronics compute power control agent technology computer vision machine learn development autonomous robots capable solve high level deliberate task natural environments approach seriously article provide essential vision learn relate aspects develop autonomous camera equip robot systemsinternet base tele operation systems utilize ubiquitous connectivity low cost bandwidth offer internet send command receive friendly feedback tele operate remote systems many scenarios efficiency task completion tactic coordination human machine multi robots require feasible several hurdle cross include internet type delay uncertainties environment uncertainties object manipulate paper propose general event base adaptive tele operation control mechanism integrate machine learn deal uncertainties mechanism test experiment tele operation soccer robots system power hybrid learn experimental result confirm mechanism present ieee robot automat soc shandong univ chinese univ hong kong chiba inst technol robot soc japan soc instrument control engn japan soc mech engn robiowant think think think think something seymour papert softbot software robot program interact software environment issue command interpret environment feedback softbots much easier build physical robots softbots attractive substrate machine learn agentarchitecture research illustrate point consider rodney unix softbot development university washington rodney use prodigy planner theo frame system represent world model custom build execution sense modules allow rodney interact unix shell real time motivation much recent work machine learn plan influence two theme ffl integrate architectures focus program combine plan learn execution capabilities laird rosenbloom mitchell suttonsmart factory research pace current decade due development many enable technologies tool available developers lead progress cyber physical systems manufacture coin cyber physical production systems ultimate goal domain integrate underlie technologies connect physical plant virtual factory real time improvement product quality process improvements predictive maintenance mass customization well mass production involve technology modules include sensor network machine learn internet things human machine interface augment reality collaborative robotics physical element research micro factory scenario envisage consist high precision micro nano position stage instal tabletop size conventional machine tool collaborative robot handle micro part run machine operations factory devices human worker supervision task due multi faceted technologies involve virtual physical systems simultaneous design strategy follow domains first flexure base micro position axis stage device design instal conventional axis desktop size mill machine secondly work zone consider effective human robot collaboration production area work zone consider social space design safe secure way help integrate devices iot',
 'direction walter potter thesis investigate techniques development mobile robot control architec tures work split two main part first part chapters simu lation technique propose potential control architectures could investigate without recourse construct physical robot second part chapter machine learn technique develop could eventually incorporate robot control architecture neither part thesis represent fully complete project rather foundations pursue two separate project could contribute toolbox development intelligent robotsdirection walter potter thesis investigate techniques development mobile robot control architectures work split two main part first part chapters simulation technique propose potential control architectures could investigate without recourse construct physical robot second part chapter machine learn technique develop could eventually incorporate robot control architecture neither part thesis represent fully complete project rather foundations pursue two separate project could contribute toolbox development intelligent robots index word',
 'application aco base algorithms data mine grow last years several supervise unsupervised learn algorithms develop use bio inspire approach recent work concern unsupervised learn focus cluster aco base techniques show great potential time new cluster techniques seek continuity data specially focus spectral base approach opposition classical centroid base approach attract increase research interest area still study aco cluster techniques work present hybrid spectral base aco cluster algorithm inspire aco cluster acoc algorithm propose approach combine acoc spectral laplacian generate new search space algorithm order obtain promise solutions new algorithm call sacoc compare well know algorithms mean spectral cluster acoc experiment measure accuracy algorithm synthetic datasets real world datasets extract uci machine learn repositoryproceed international conference ants brussels belgium september final publication available springer via http doi org application aco base algorithms data mine grow last years several supervise unsupervised learn algorithms develop use bio inspire approach recent work concern unsupervised learn focus cluster show great potential aco base techniques work present aco base cluster algorithm inspire aco cluster acoc algorithm propose approach restructure acoc centroid base technique medoid base technique properties search space necessarily know instead rely information distance amongst data new algorithm call macoc compare well know algorithms mean partition around medoids acoc experiment measure accuracy algorithm synthetic datasets real world datasets extract uci machine learn repository work partly support spanish ministryof science education project tin savier airbusdefense space project fuam fuam',
 'paper describe methodologies apply result achieve framework esprit basic research action learn project learn one rst project work towards application machine learn techniques elds industrial relevance much complex domains usually treat research particular learn aim ease program robots enhance ability cooperate humans paper give short introduction learn robotics three applications consideration learn afterwards learn methodologies use applications experimental setups result obtain describe general find provide good examples good interface learn performance components crucial success extension quot program demonstration quot paradigm robotics become one key aspects learn',
 'currently robot control systems specifically design engineer fine tune particular problems particular robots lead significant waste man hours engineer phd level work implement reimplement adapt controllers similar task different robots result inefficient robotics industry whole thus need automate least semi automate controller reusability arise project investigate hurdle need overcome attain controller universality look possible methods bootstrap controllers different robot sensors actuators case study conduct perform migration controller wheel robot mobile vision system pioneer robot legged robot mobile head mount camera aldebaran nao two robots different modalities make task challenge mean two different robots perform task machine learn methods deploy use artificial neural network ann learn entire sensor decision system robot motor api tree leave sensor feature extraction low level motor control hand engineer method work reasonably well effectively link number controllers design pioneer onto nao sensors actuators preliminary methods provide insight future prospect robots program learn help humansnowadays large amount data automatically generate devices sensors measure instance parameters production process environmental condition transport goods energy consumption smart home traffic volume air pollution water consumption pulse blood pressure individuals collection transmission data enable electronics software sensors network connectivity embed physical object object infrastructure connect object call internet things iot already billion devices connect iot number twice large world population time iot provide data physical environment level detail never know human history understand data create opportunities improve way live learn work entertain instance information obtain data analysis modules embed exist process could help optimization lead sustainable systems save resources sectors manufacture logistics energy utilities public sector healthcare iot inherent distribute nature resource constraints dynamism network participants well amount diverse type data collect challenge even advance automate data analysis methods know today currently strong research focus centralization data cloud process accord paradigm parallel high performance compute however resources devices sensors data generate side might suffice transmit data instance pervasive distribute systems wireless sensors network highly communication constrain stream high throughput applications data mass simply huge send exist communication line like satellite connections hence iot require new generation distribute algorithms resource aware intelligently reduce amount data transmit process throughout analysis chain thesis deal distribute analysis vertically partition sensor measurements communication constraints particularly challenge scenario observations distribute nod feature value learn accurate prediction model may require combination information different nod necessarily lead communication main question design communication efficient algorithms scenario time preserve sufficient accuracy first part ',
 'introduce analyze new algorithm linear classification combine rosenblatt perceptron algorithm helmbold warmuth leave one method like vapnik maximal margin classifier algorithm take advantage data linearly separable large margins compare vapnik algorithm however much simpler implement much efficient term computation time also show algorithm efficiently use high dimensional space use kernel function perform experiment use algorithm variants classify image handwritten digits performance algorithm close good performance maximal margin classifiers problem save significantly computation time program effort introduction one influential developments theory machine learn last years vapnik work supp',
 'volume include extend revise versions paper present workshop combinations intelligent methods applications cima intend become forum exchange experience ideas among researchers practitioners deal combinations different intelligent methods artificial intelligence aim create integrate hybrid methods benefit components exist present efforts combine soft compute methods fuzzy logic neural network genetic algorithms another stream efforts integrate case base reason machine learn soft compute methods combinations widely explore like neuro symbolic methods neuro fuzzy methods methods combine rule base case base reason cima hold conjunction ieee international conference tool artificial intelligence ictai',
 'izbolj evanje obstoje proizvodnje obdelovalnih sistemov zahteva nenehno posodabljanje integracijo najnovej tehnologij proizvodne sisteme proizvodnih spremenljivk edalje tem pove uje mno ica podatkov moramo obdelati velikokrat klasi analiti metode optimizacije odpovedo zaradi tega smo prisiljeni bolje izkoristiti razpolo ljive proizvodne vire zato moramo pose naprednej pristopih evanja problemov evanje zahtevnih problemov edalje pogosteje uporabljajo razli podro umetne inteligence zlasti strojnega enja pregled sedaj opravljenih raziskav pokazal obstoje razviti sistemi precej ozko usmerjeni disertaciji predlagamo popolnoma nov pristop modeliranju cnc obdelav pomo novega gravitacijskega iskalnega algoritma gsa spada med metode skupinske inteligence razviti inteligentni sistem deluje osnovi osnovnih newtonovih fizikalnih zakonov oziroma osnovi interakcij med masnimi telesi prostoru primerjavo potrditev ustreznosti rezultatov doktorske disertacije smo uporabili tudi metodo modeliranja rojem delcev pso primerjava pokazala gsa algoritem primeren modeliranje obdelav odrezovanjem saj odstopanja eksperimentalnih podatkov sprejemljivih mejah dobljeni modeli dobro opisali postopek odrezovanja materiala stru enjem smo uporabili kot postopek odrezovanja posebej velja omeniti gsa algoritem najslab primeru vsaj dvakrat hitrej enakovrednega pso algoritma dobljen model cnc obdelave smo nato uporabili kriterijsko optimiranje obdelovalnih parametrov optimalne hrapavosti obdelane povr ine rezalnih sil asovne obstojnosti orodja vsaka izmed omenjenih odvisnih spremenljivk prispeva optimalnemu delovanju cnc obdelovalnega stroja kar zni uje stro proizvodnje kriterijsko optimiranje smo izvedli pomo nsga algoritma optimiranje smo morali dolo iti tudi omejitve smo dolo ili pomo teoreti nih izra unov jih preverili pomo eksperimentalnih podatkov zaradi obsega dela smo omejili stru enje hkrati delu predstavljene osnove prilagoditev uporabo metod ostalih obdelovalnih strojih saj predlagan pristop ']


In [ ]:
# I had to give the topics of each paper because evaluating abstracts is hard 
# So this code produces a list with all the clusters, and which can be further used to evaluate the performance of this program

import re
length2 = len(final_list)
topicss = []
for i in range(length2):
    topicss.append([])
for i in range(len(final_list)):
    
    for j in final_list[i]:
        count = 0
        for k in range(len(data_list2)):
          if j == data_list2[k]:
            topicss[i].append(topic_list2[k])
          else:
            count+=1
        
       
        


topicss

[['Adversarial Scene Editing Automatic Object Removal from Weak Supervision',
  'Dialogbased Interactive Image Retrieval',
  'Generalizing Graph Matching beyond Quadratic Assignment Model',
  'Greedy Hash Towards Fast Optimization for Accurate Hash Coding in CNN',
  'HighResolution Image Inpainting using MultiScale Neural Patch Synthesis'],
 ['A Block Coordinate Ascent Algorithm for MeanVariance Optimization',
  'A Deep Bayesian Policy Reuse Approach Against NonStationary Agents',
  'A LinearTime Kernel GoodnessofFit Test',
  'A Neural Compositional Paradigm for Image Captioning',
  'A Reduction of Imitation Learning and Structured Prediction to NoRegret Online Learning',
  'A Test of Relative Similarity For Model Selection in Generative Models',
  'A Unified Feature Disentangler for MultiDomain Image Translation and Manipulation',
  'A2Nets Double Attention Networks',
  'Active Learning for NonParametric Regression Using Purely Random Trees',
  'Adversarial vulnerability for any class

In [ ]:
!pip install yake

     |████████████████████████████████| 61kB 3.4MB/s 
     |████████████████████████████████| 92kB 5.2MB/s 
  Created wheel for segtok: filename=segtok-1.5.10-cp37-none-any.whl size=25019 sha256=28dc7666d1302593226798930fed0a23750e78d236b681b2febb44e919bccf3e
  Stored in directory: /root/.cache/pip/wheels/b4/39/f6/9ca1c5cabde964d728023b5751c3a206a5c8cc40252321fb6b
Successfully built segtok


In [ ]:
import yake
kw_extractor = yake.KeywordExtractor()
language = "en"
max_ngram_size = 2
deduplication_threshold = 0.1
numOfKeywords = 20
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)

cluster_append = []
length = len(appended_final_list)
for i in range(length):
  cluster_append.append([])
for i in range(len(appended_final_list)):
  keywords = custom_kw_extractor.extract_keywords(appended_final_list[i])
  for k,w in keywords[:5]:
    cluster_append[i].append(k)

In [ ]:
len(appended_final_list)

15

In [ ]:
# Finding the most occurred keywords of each cluster


# from collections import Counter

# cluster_append = []
# for i in appended_final_list:
  
#   split_it = i.split()
 

#   Countervar = Counter(split_it)
 

#   most_occur = Countervar.most_common(4)

#   cluster_append.append(most_occur)
  


In [ ]:
for i in range(len(topicss)):
  topicss[i].insert(0,cluster_append[i])

In [ ]:
topicss

In [ ]:
# Creating the graph of the clusters for USER UNDERSTANDING

from graphviz import Digraph

g = Digraph('G', filename='process.gv', engine='sfdp')




for i in range(len(final_list)):
  

  
  g.edge('Total Samples =  {amount} \n Clusters defined'.format(amount = len(data_list2)), 'Cluster '+str(i)+'\n{len}'.format(len = len(final_list[i]))+'\n'+str(cluster_append[i]))

  g.graph_attr['rankdir'] = 'LR'
  g.edge_attr.update(arrowhead='vee', arrowsize='0.5',)





g.view()

'process.gv.pdf'